In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 
import pickle
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/data/huggingface/hub/sentence-transformers_all-MiniLM-L6-v2')
import oracledb
connection = oracledb.connect(user="", password="", host="",port=,service_name="")

import oracledb
from sqlalchemy import create_engine
import cx_Oracle
from tqdm import tqdm

from sentence_transformers.util import cos_sim
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from itertools import chain

from tqdm import tqdm
import re
import os

def get_dict(x,dictionary):
    return [dictionary[str(item)] for item in x]
def get_larger(x,number):
    return [item for item in x if item>number]
def compare_larger(x,number):
    return any([item>number for item in x])

In [3]:
query = 'SELECT t.* FROM "biotechnology_paper_author" t '
df_scopus=pd.read_sql(query, connection)
# df_scopus=pd.read_pickle('data/df_scopus.pickle')
df_scopus

/tmp/ipykernel_570228/2801714898.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_scopus=pd.read_sql(query, connection)


,eid,author_names,affilname,author_ids,afid
0,2-s2.0-84991318817,"Kamthania, Mohit;Sharma, D. K.","JH Government Post Graduate College, Betul;Man...",55479937900;57206773481,60097662;60094550;60009689
1,2-s2.0-84982787473,"Ghosh, Tuhin;Basu, Ankita;Adhikari, Dipan;Roy,...",Acharya Brojendra Nath Seal College;Hooghly Mo...,57197638841;7402167211;57213724746;57193908340...,60099701;60095158;60030482;100318031
2,2-s2.0-84988386029,"Kagithoju, Srikanth;Godishala, Vikram;Nanna, R...",Kakatiya University,55584286700;55584120800;39762350700,60023968
3,2-s2.0-84983779033,"Surendhiran, Duraiarasan;Sirajunnisa, Abdul Ra...",Annamalai University,55735347700;56034893200;24468826200,60027171
4,2-s2.0-85016090291,"Shahbaaz, Mohd;Ahmad, Faizan;Imtaiyaz Hassan, Md",Jamia Millia Islamia,56039303200;56108326000;7401625549,60020458
...,...,...,...,...,...
29442718,2-s2.0-0022312055,"Kuskowska-Wolk, A.;Puncewicz, B.",Zakladu Szkolenia i Oswiaty Zdrowotnej Instytu...,6602097024;6603405065,120108634.0
29442719,2-s2.0-0022295156,"Kuskowska-Wolk, A.;Ilinska-Debniak, K.",Zakladu Szkolenia i Oswiaty Zdrowotnej Instytu...,6602097024;6506557146,120108634.0
29442720,2-s2.0-0021857140,"Gniot-Szulzycka, J.;Korzycka, W.;Stefanska, J.",Uniwersytet Mikołaja Kopernika w Toruniu,6701780216;6504020998;6603414444,60021858.0
29442721,2-s2.0-85142401310,"Serrano Albal, María;Silvestri, Giuseppe;Kiazi...",JSR Genetics Ltd;Topigs Norsvin;Canterbury Chr...,57220602738;57196370255;57202201190;5722655303...,60120852;60115175;60019444;60010818


In [4]:
# df_scopus.to_pickle('data/df_scopus.pickle')

In [5]:
# t1=df_scopus['author_names'].str.split(';',expand=True).stack()
# t2=df_scopus['author_ids'].str.split(';',expand=True).stack()
# t3=pd.DataFrame(t2).merge(pd.DataFrame(t1),how='left',left_index=True,right_index=True)
# t3.to_pickle('data/t3.pickle')

In [7]:
t3=pd.read_pickle('data/t3.pickle')
scopus_author_id={}
for author_id,author_name in zip(t3['0_x'],t3['0_y']):
    scopus_author_id[author_id]=author_name
del t3

In [8]:
def fine_name(x):
    if ',' not in x:
        name_list=x.split(' ')
        b=name_list[-1]
        name_list.remove(name_list[-1])
        name_list=' '.join(name_list)
        name_list=b+', '+name_list
        return name_list
    else:
        return x
fine_name('ss, sss ssss')

'ss, sss ssss'

In [9]:
fine_name('ss, sss ssss')

'ss, sss ssss'

In [10]:
query = 'SELECT t.* FROM "biotechnology_author" t '
# df_patstat=pd.read_sql(query, connection)
df_patstat=pd.read_pickle('data/df_patstat.pickle')
df_patstat

,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address
8,15763360,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
11,15768652,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
15,15772733,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
24,15972133,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
26,15978103,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
...,...,...,...,...,...,...,...
40251181,567558584,85453972.0,0.0,"SHANGYAOTANG BIG HEALTH INDUSTRY GROUP CO., LTD.",185453972.0,UNKNOWN,None
40251187,557454227,85454359.0,5.0,SHE KEPENG,185454359.0,UNKNOWN,None
40251190,564771023,85438757.0,1.0,JAMES O. DAVIS,185438757.0,UNKNOWN,None
40251192,571323564,85440714.0,1.0,Richard Gagliardo,185440714.0,UNKNOWN,None


In [11]:
# df_patstat.to_pickle('data/df_patstat.pickle')

In [12]:
df_record=pd.read_pickle('data/df_record.pickle')
with open('labels.pickle', 'rb') as f:
    labels=pickle.load(f)

df_record['labels']=labels
df_record=df_record[df_record['labels']>100]
df_record

,appln_id,title,eid,doi,source_id,title_embedding,labels
26,4524991.0,HUMAN KUNITZ-TYPE PROTEASE INHIBITORS,NaN,NaN,NaN,"[-0.026624046, -0.059168316, -0.08670005, -0.0...",2258
31,4592011.0,25-HYDROXYVITAMIN D3-1.ALPHA.-HYDROXYLASE AND ...,NaN,NaN,NaN,"[-0.056475606, -0.06453719, -0.009920548, -0.0...",1822
49,15567268.0,ISOLATED SEQUENCES OF DNA ENCODING L1 AND L2 C...,NaN,NaN,NaN,"[-0.07865718, 0.027101953, -0.050718345, -0.04...",199
51,15461030.0,HUMAN KUNITZ-TYPE PROTEASE INHIBITOR VARIANTS,NaN,NaN,NaN,"[-0.004485598, -0.072244026, -0.068950705, -0....",2258
62,16628206.0,NOVEL PLASMID AND USE THEREOF,NaN,NaN,NaN,"[-0.12733708, -0.050047275, -0.024822485, -0.0...",4373
...,...,...,...,...,...,...,...
29545065,NaN,PTEN expression in human cumulus cells is asso...,2-s2.0-85128146377,10.1017/S096719942200003X,19073.0,"[-0.05939561, -0.06043713, -0.030064046, -0.04...",3368
29545068,NaN,Comparison of pregnancy rate in dromedary came...,2-s2.0-85128135701,10.1017/S096719942100099X,19073.0,"[-0.016140513, -0.0016672981, -0.050634366, 0....",3369
29545070,NaN,The expression profile of WNT/β-catanin signal...,2-s2.0-85128119080,10.1017/S0967199422000028,19073.0,"[-0.009771967, -0.065358974, -0.070520185, 0.1...",3090
29545075,NaN,Relative expression of the developmentally imp...,2-s2.0-85125811955,10.1017/S0967199421000976,19073.0,"[-0.0929608, -0.021178935, -0.072981216, -0.01...",3356


In [13]:
df_patstat=df_patstat[df_patstat['appln_id'].isin(df_record['appln_id'])]
df_patstat

,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address
8,15763360,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
11,15768652,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
15,15772733,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
24,15972133,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
26,15978103,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
...,...,...,...,...,...,...,...
40251181,567558584,85453972.0,0.0,"SHANGYAOTANG BIG HEALTH INDUSTRY GROUP CO., LTD.",185453972.0,UNKNOWN,None
40251187,557454227,85454359.0,5.0,SHE KEPENG,185454359.0,UNKNOWN,None
40251190,564771023,85438757.0,1.0,JAMES O. DAVIS,185438757.0,UNKNOWN,None
40251192,571323564,85440714.0,1.0,Richard Gagliardo,185440714.0,UNKNOWN,None


In [14]:
print('公司总数',len(df_patstat[df_patstat['psn_sector']=='COMPANY']['psn_id'].unique()))

公司总数 221921


In [17]:
df_patstat_1=df_patstat.merge(df_record[['title','labels','appln_id']],how='inner',left_on='appln_id',right_on='appln_id')
df_scopus_1=df_scopus.merge(df_record[['title','labels','eid','doi']],how='inner',left_on='eid',right_on='eid')
df_patstat_1=df_patstat_1[df_patstat_1['person_id'].notnull()]
df_scopus_1=df_scopus_1[df_scopus_1['author_ids'].notnull()]
df_scopus_1

,eid,author_names,affilname,author_ids,afid,title,labels,doi
0,2-s2.0-84988386029,"Kagithoju, Srikanth;Godishala, Vikram;Nanna, R...",Kakatiya University,55584286700;55584120800;39762350700,60023968,Eco-friendly and green synthesis of silver nan...,2095,10.1007/s13205-014-0272-3
1,2-s2.0-85016090291,"Shahbaaz, Mohd;Ahmad, Faizan;Imtaiyaz Hassan, Md",Jamia Millia Islamia,56039303200;56108326000;7401625549,60020458,Structure-based functional annotation of putat...,4542,10.1007/s13205-014-0231-z
2,2-s2.0-84955299251,"Srinath, B. S.;Ravishankar Rai, V.",University of Mysore,56524425300;8341550600,60013290,"Biosynthesis of highly monodispersed, spherica...",2095,10.1007/s13205-014-0265-2
3,2-s2.0-85016080990,"Jaiswal, Manjit;Dudhe, Rupesh;Sharma, P. K.",Galgotias University,57193906287;35097089900;57212203890,60120915,Nanoemulsion: an advanced mode of drug deliver...,4297,10.1007/s13205-014-0214-0
4,2-s2.0-85013482257,"Prajapati, Vimal S.;Trivedi, Ujjval B.;Patel, ...",Sardar Patel University,55606368400;8699184200;35498547500,60013979,A statistical approach for the production of t...,4648,10.1007/s13205-014-0213-1
...,...,...,...,...,...,...,...,...
9378398,2-s2.0-85142401803,"Veraguas-Dávila, Daniel;Saéz-Ruíz, Darling;Álv...",Universidad de Concepcion,57226836109;57204547787;57474339200;6701773159...,60001282,Analysis of trophectoderm markers in domestic ...,244,10.1017/S096719942200034X
9378399,2-s2.0-85129010913,"Müller, M. C.;Monte, A. P.O.;Lins, T. L.B.G.;M...",Universidade Federal Rural do Semi-Árido – UFE...,57189628022;56989006400;55832614800;5583289940...,60033203;128052677;110735077,Fibroblast growth factor-2 promotes in vitro a...,3402,10.1017/S0967199421000964
9378400,2-s2.0-85142401643,"Sciorio, Romualdo;Tramontano, Luca;Bellaminutt...",University of Ferrara;Hôpitaux Universitaires ...,35389930700;57213154806;56902530900;5653161470...,60024690;60009963;60002975;128875271;128875162,Review of the impact of COVID-19 on male repro...,421,10.1017/S0967199421000666
9378401,2-s2.0-85128146377,"Yao, Jianfeng;Li, Ming;Lin, Liqing;Li, Youzhu;...",The First Affiliated Hospital of Xiamen Univer...,56965937800;57199160334;57574494100;5719241089...,60122372;60017060;60005206;114105372;112938123,PTEN expression in human cumulus cells is asso...,3368,10.1017/S096719942200003X


In [18]:
for col in ['appln_id','person_id','psn_id']:
    print(col)
    df_patstat_1[col]=df_patstat_1[col].astype(int).astype(str)

appln_id
person_id
psn_id


In [19]:
df_scopus_1.reset_index(drop=True,inplace=True)

In [20]:
max(df_record['labels'])

4778

In [21]:
df_scopus_1=df_scopus_1[df_scopus_1['author_names'].notnull()]
df_patstat_1=df_patstat_1[df_patstat_1['psn_id'].notnull()]

In [22]:
del df_scopus
del df_patstat
del t3

# 开始scopus1和patstat1

In [17]:
df_patstat_1=pd.read_pickle('data/df_patstat_1.pickle')
print('df_patstat_1读取完毕')
df_scopus_1=pd.read_pickle('data/df_scopus_1.pickle')
print('df_scopus_1读取完毕')

df_patstat_1读取完毕
df_scopus_1读取完毕


In [18]:
# scopus的author_id映射author_name
# patstata的PSN_ID和PSN_NAME的映射
t3=pd.read_pickle('data/t3.pickle')
scopus_author_id={}
for author_id,author_name in zip(t3['0_x'],t3['0_y']):
    scopus_author_id[author_id]=author_name
del t3

patent_author_id={id:name.title().strip() for id,name in zip(df_patstat_1['psn_id'],df_patstat_1['psn_name'])}

# scopus_id_author={v:k for k,v in scopus_author_id.items()}
# patent_id_author={v:k for k,v in patent_author_id.items()}

In [19]:
scopus_author_id['56965937800']

'Yao, Jianfeng'

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# bert_tokenizer = AutoTokenizer.from_pretrained('/data/huggingface/hub/dslim-bert-large-NER')
# bert_model = AutoModelForTokenClassification.from_pretrained('/data/huggingface/hub/dslim-bert-large-NER')

# nlp = pipeline('ner', model=bert_model, tokenizer=bert_tokenizer, aggregation_strategy='max')

# def get_person_name(x):
#     ner_list=nlp(x)
#     if len([i for i in ner_list if i.get('entity_group')=='PER'])>0:
#         return x[0:max([i.get('end') for i in ner_list if i.get('entity_group')=='PER'])]
#     else:
#         return None
# get_person_name('Fersht, Alan, Mrc Unit For Protein Function And')

In [ ]:
len(patent_author_id) 

In [ ]:
total=0
for label in tqdm(range(101,4779)):
    df_scopus_name=df_scopus_1[df_scopus_1['labels']==label][['eid','doi','title', 'author_ids', 'labels']]
    df_scopus_name['author_id']=df_scopus_name['author_ids'].str.split(';')
    df_scopus_name=df_scopus_name.explode(['author_id'])
    # df_scopus_name=df_scopus_name.drop_duplicates('author_id')

    df_patstat_name=df_patstat_1[df_patstat_1['labels']==label]
    df_patstat_name=df_patstat_name[df_patstat_name['psn_id'].notnull()]
    df_patstat_name=df_patstat_name.drop_duplicates('psn_id')

    #     print(label,'开始计算')
    # # 计算出相似性矩阵
    author_id_matching=df_scopus_name['author_id'].tolist()
    psn_id_matching=df_patstat_name['psn_id'].astype(str).tolist()
    appln_id_matching=df_patstat_name['appln_id'].tolist()
    eid_matching=df_scopus_name['eid'].tolist()
    doi_matching=df_scopus_name['doi'].tolist()
    scopus_matching_name=[scopus_author_id[i] for i in author_id_matching]
    psn_id_matching_name=[patent_author_id[i] for i in psn_id_matching]
    title_scopus_matching=df_scopus_name['title'].tolist()
    title_patstat_matching=df_patstat_name['title'].astype(str).tolist()
    if len(psn_id_matching_name)>0 and len(author_id_matching)>0:
    #         # 首先，新建一个关于scopus所有作者id的字典，用来更新它所对应的patstat作者id
    #         scopus_name_pair_90={k:[] for k in df_scopus_name['author_ids'].unique()}
    #         scopus_name_pair_95={k:[] for k in df_scopus_name['author_ids'].unique()}
        scopus_name_pair_80={k:[] for k in df_scopus_name['author_id'].unique()}

        embeddings_scopus = model.encode(scopus_matching_name,show_progress_bar=False)
        embeddings_patstat = model.encode(psn_id_matching_name,show_progress_bar=False)
        scores=cos_sim(embeddings_scopus,embeddings_patstat)
    #         print(label,'开始更新id关系')
        # 把每一条的scopus作者id，所有可以配得上的patstat作者id，都配为一人，
        # 因为假设scopus的数据是准确的，而psn的数据是存在歧义的。
        for row,score in tqdm(enumerate(scores)):#row是论文的索引，score是专利的数量，内容是专利的相似分数
            if len(np.where(score>0.80)[0])>0:
                cols=np.where(score>0.80)[0]  #cols是第几个专利相似度达标了

                scopus_name_pair_80[author_id_matching[row]].extend([[scopus_matching_name[row],
                                                                   psn_id_matching[idx],
                                                                   appln_id_matching[idx],
                                                                   psn_id_matching_name[idx],
                                                                  eid_matching[row],
                                                                  doi_matching[row],
                                                                 title_scopus_matching[row],
                                                                  title_patstat_matching[idx],
                                                                    score[idx].numpy(),
                                                                  ] for idx in cols])
#                 print(author_id_matching[row],'----',[[scopus_matching_name[row],
#                                                                    psn_id_matching[idx],
#                                                                    appln_id_matching[idx],
#                                                                    psn_id_matching_name[idx],
#                                                                   eid_matching[row],
#                                                                   doi_matching[row],
#                                                                  title_scopus_matching[row],
#                                                                   title_patstat_matching[idx],
#                                                                     score[idx].numpy(),
#                                                                   ] for idx in cols])
        df_scopus_name['match']=df_scopus_name['author_id'].map(scopus_name_pair_80)
        df_scopus_name=df_scopus_name[df_scopus_name['match'].apply(lambda x:len(x)>0)]
        df_scopus_name.to_pickle('data/df_matchs_version2/df_match_{}.pickle'.format(str(label)))

In [ ]:
# df_patstat_1.to_pickle('data/df_patstat_1.pickle')
# print('df_patstat_1存储完毕')
# df_scopus_1.to_pickle('data/df_scopus_1.pickle')
# print('df_scopus_1存储完毕')

In [175]:
df_match_result=pd.concat([pd.read_pickle('data/df_matchs_version2/'+i) for i in os.listdir('data/df_matchs_version2')])
df_match_result=df_match_result.explode(['match'])
df_match_result['psn_id_matching']=df_match_result['match'].apply(lambda x:x[1])
df_match_result['psn_id_matching_name']=df_match_result['match'].apply(lambda x:x[3])
df_match_result['scopus_matching_name']=df_match_result['match'].apply(lambda x:x[0])
df_match_result['appln_id_matching']=df_match_result['match'].apply(lambda x:x[2])
df_match_result['eid_matching']=df_match_result['match'].apply(lambda x:x[4])
df_match_result['doi_matching']=df_match_result['match'].apply(lambda x:x[5])
df_match_result['title_scopus_matching']=df_match_result['match'].apply(lambda x:x[6])
df_match_result['title_patstat_matching']=df_match_result['match'].apply(lambda x:x[7])
df_match_result['score_matching']=df_match_result['match'].apply(lambda x:x[8])
del df_match_result['match']
df_match_result

,eid,doi,title,author_ids,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching
1378743,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,57199705592;7004114700;7401548455;7101829857;7...,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.984326
1378743,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,57199705592;7004114700;7401548455;7101829857;7...,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.978623
1378743,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,57199705592;7004114700;7401548455;7101829857;7...,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.972987
8485011,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,54961880400;57224168054;57188758633;1933813810...,4387,19338138100,29535451,Shen Jian,"Shen, You Cheng",479402407,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Elm flavor water caltrop vinegar brewing method,0.817428
8485011,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,54961880400;57224168054;57188758633;1933813810...,4387,56687931500,34569678,Wang Ling,"Wang, Chin Kun",267426954,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Method for preparing pineapple peel residue fr...,0.817774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285422,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,55876382800;57244440000,2189,57244440000,19754519,"Liu, Kejing","Liu, Yang",413343242,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Hot air type scald instrument,0.806795
9285422,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,55876382800;57244440000,2189,57244440000,36171110,Yang Jianhua,"Liu, Yang",331512701,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Multi-functional electric heater,0.818462
9285422,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,55876382800;57244440000,2189,57244440000,19738704,Liu Yiyang,"Liu, Yang",443602139,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Semiconductor refrigeration head hoop,0.850789
7820957,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,56325243000;7401710943;7401959793,1192,56325243000,4759227,Cheng Lu,"Cheng, L. C.",520520428,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,Surgical dressing change frame,0.812837


In [21]:
df_match_result.columns

Index(['eid', 'doi', 'title', 'author_ids', 'labels', 'author_id',
       'psn_id_matching', 'psn_id_matching_name', 'scopus_matching_name',
       'appln_id_matching', 'eid_matching', 'doi_matching',
       'title_scopus_matching', 'title_patstat_matching', 'score_matching'],
      dtype='object')

In [176]:
df_match_result=df_match_result[['labels', 'author_id', 'psn_id_matching', 'psn_id_matching_name', 'scopus_matching_name',
       'appln_id_matching', 'eid_matching', 'doi_matching', 'title_scopus_matching', 'title_patstat_matching', 'score_matching']]
df_match_result=df_match_result.drop_duplicates()
df_match_result

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching
1378743,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.984326
1378743,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.978623
1378743,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.972987
8485011,4387,19338138100,29535451,Shen Jian,"Shen, You Cheng",479402407,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Elm flavor water caltrop vinegar brewing method,0.817428
8485011,4387,56687931500,34569678,Wang Ling,"Wang, Chin Kun",267426954,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Method for preparing pineapple peel residue fr...,0.817774
...,...,...,...,...,...,...,...,...,...,...,...
9285422,2189,57244440000,19754519,"Liu, Kejing","Liu, Yang",413343242,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Hot air type scald instrument,0.806795
9285422,2189,57244440000,36171110,Yang Jianhua,"Liu, Yang",331512701,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Multi-functional electric heater,0.818462
9285422,2189,57244440000,19738704,Liu Yiyang,"Liu, Yang",443602139,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Semiconductor refrigeration head hoop,0.850789
7820957,1192,56325243000,4759227,Cheng Lu,"Cheng, L. C.",520520428,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,Surgical dressing change frame,0.812837


In [179]:
len(df_match_result['psn_id_matching'].unique())

354730

In [9]:
df_match_result[['author_id']].drop_duplicates().to_csv('data/df_match_result_author_id.csv',index=None,encoding='utf-8-sig')

# 加机构信息

## 学术机构信息

In [23]:
query = 'SELECT t.* FROM "df_match_result_author_id_aff" t '
df_match_result_author_id_aff=pd.read_sql(query, connection)
df_match_result_author_id_aff
# df_scopus=pd.read_pickle('data/df_scopus.pickle')

/tmp/ipykernel_1014740/681816059.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_match_result_author_id_aff=pd.read_sql(query, connection)


,author_id,eid,author_name,author_afid
0,57210572188,2-s2.0-85085881634,"Zhao, Zhihao",60083514
1,56029200100,2-s2.0-85085881634,"Jin, Longfei",60083514
2,35398019900,2-s2.0-85085771934,"Thyagarajan, S. P.",60023483
3,57193761775,2-s2.0-85085700493,"Fan, Guangsen",60022528
4,57217005974,2-s2.0-85085700493,"Cheng, Liujie",60022528
...,...,...,...,...
21055546,56296323900,2-s2.0-85024405058,"Kim, Eun Young",116760088
21055547,26021408600,2-s2.0-85024405058,"Park, Se Pill",60117634
21055548,26021408600,2-s2.0-85024405058,"Park, Se Pill",116760088
21055549,55959433600,2-s2.0-85024378032,"Guo, Qing",60015003


In [24]:
query = 'SELECT t.* FROM "df_match_result_author_id_affflia" t '
df_match_result_author_id_affflia=pd.read_sql(query, connection)
df_match_result_author_id_affflia
# df_scopus=pd.read_pickle('data/df_scopus.pickle')

/tmp/ipykernel_1014740/1238954063.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_match_result_author_id_affflia=pd.read_sql(query, connection)


,afid,affiliation_city,affiliation_country,affilname
0,60032083,Buffalo,United States,"University at Buffalo, The State University of..."
1,60030452,San Marcos,United States,Texas State University
2,60018922,Seattle,United States,Children's Hospital and Regional Medical Center
3,60032653,Poitiers,France,Universite de Poitiers
4,60072481,Johnson,United States,Quillen-Dishner College of Medicine
...,...,...,...,...
305885,122369171,None,China,Jiangxi Province Research Center of Extracts a...
305886,119154558,None,China,School of Applied and Chemical Engineering of ...
305887,127703075,Nanyang,China,Ltd.
305888,124918410,None,China,Tongliao Institute for Endemic Disease Control...


In [25]:
df_match_result_author_id_aff=df_match_result_author_id_aff.merge(df_match_result_author_id_affflia,how='left',left_on='author_afid',right_on='afid')
df_match_result_author_id_aff

,author_id,eid,author_name,author_afid,afid,affiliation_city,affiliation_country,affilname
0,57210572188,2-s2.0-85085881634,"Zhao, Zhihao",60083514,60083514,Haikou,China,Chinese Academy of Tropical Agricultural Sciences
1,57210572188,2-s2.0-85085881634,"Zhao, Zhihao",60083514,60083514,Danzhou,China,Chinese Academy of Tropical Agricultural Sciences
2,56029200100,2-s2.0-85085881634,"Jin, Longfei",60083514,60083514,Haikou,China,Chinese Academy of Tropical Agricultural Sciences
3,56029200100,2-s2.0-85085881634,"Jin, Longfei",60083514,60083514,Danzhou,China,Chinese Academy of Tropical Agricultural Sciences
4,35398019900,2-s2.0-85085771934,"Thyagarajan, S. P.",60023483,60023483,Chennai,India,Sri Ramachandra Institute of Higher Education ...
...,...,...,...,...,...,...,...,...
40802820,56296323900,2-s2.0-85024405058,"Kim, Eun Young",116760088,116760088,Seoul,South Korea,Mirae Cell Bio
40802821,26021408600,2-s2.0-85024405058,"Park, Se Pill",60117634,60117634,Jeju,South Korea,Jeju National University
40802822,26021408600,2-s2.0-85024405058,"Park, Se Pill",116760088,116760088,Seoul,South Korea,Mirae Cell Bio
40802823,55959433600,2-s2.0-85024378032,"Guo, Qing",60015003,60015003,Yanji,China,Yanbian University


In [26]:
df_match_result_author_id_aff=df_match_result_author_id_aff.drop_duplicates(['author_id','afid','affiliation_city','affiliation_country','affilname'])
df_match_result_author_id_aff

,author_id,eid,author_name,author_afid,afid,affiliation_city,affiliation_country,affilname
0,57210572188,2-s2.0-85085881634,"Zhao, Zhihao",60083514,60083514,Haikou,China,Chinese Academy of Tropical Agricultural Sciences
1,57210572188,2-s2.0-85085881634,"Zhao, Zhihao",60083514,60083514,Danzhou,China,Chinese Academy of Tropical Agricultural Sciences
2,56029200100,2-s2.0-85085881634,"Jin, Longfei",60083514,60083514,Haikou,China,Chinese Academy of Tropical Agricultural Sciences
3,56029200100,2-s2.0-85085881634,"Jin, Longfei",60083514,60083514,Danzhou,China,Chinese Academy of Tropical Agricultural Sciences
4,35398019900,2-s2.0-85085771934,"Thyagarajan, S. P.",60023483,60023483,Chennai,India,Sri Ramachandra Institute of Higher Education ...
...,...,...,...,...,...,...,...,...
40802774,7005358889,2-s2.0-47849093353,"Sayaka, Wakayama",60015540,60015540,Tokyo,Japan,Japan Society for the Promotion of Science
40802793,51565382000,2-s2.0-42149093157,"Zhang, Yunhai",60007971,60007971,Frederiksberg,Denmark,mica e Biol&oacute
40802794,51565382000,2-s2.0-42149093157,"Zhang, Yunhai",60007971,60007971,Frederiksberg,Denmark,Det Natur- og Biovidenskabelige Fakultet
40802799,56236132300,2-s2.0-84937073711,"Wu, Juan",60277181,60277181,Changsha,China,Central South University Xiangya School of Med...


In [27]:
df_match_result_author_id_aff_agg=df_match_result_author_id_aff.groupby(['author_id']).agg(author_name=('author_name',list),
                                                                                          afid=('afid',list),
                                                                                           affiliation_city=('affiliation_city',list),
                                                                                           affiliation_country=('affiliation_country',list),
                                                                                           affilname=('affilname',list),
                                                                                          ).reset_index()
df_match_result_author_id_aff_agg

,author_id,author_name,afid,affiliation_city,affiliation_country,affilname
0,10038789900,"[Liang, Changyong, Liang, Changyong, Liang, Ch...","[60002836, 60001604, 60001604, 111155536]","[Hefei, Beijing, Beijing, Hefei]","[China, China, China, China]","[Hefei University of Technology, Ministry of E..."
1,10038814800,"[Suzuki, Shugo, Suzuki, Shugo, Suzuki, Shugo, ...","[60004321, 60172319, 60024705, 60003608, 60271...","[Nagoya, Osaka, Nagoya, Nagoya, Nagoya, Aichi ...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...",[Nagoya City University Graduate School of Med...
2,10038841100,"[Ang, Wee Han, Ang, Wee Han, Wee, Han Ang, Wee...","[60017161, 60017161, 60022195, 60022195, 60022...","[Singapore City, Singapore City, Cambridge, Ca...","[Singapore, Singapore, United States, United S...","[National University of Singapore, CSIC - Cent..."
3,10038844100,"[Jiao, Lijuan, Jiao, Lijuan, Jiao, Lijuan, Jia...","[60009559, 60001604, 60001604, 60019118, 60007...","[Wuhu, Beijing, Beijing, Hefei, Baton Rouge]","[China, China, China, China, United States]","[Anhui Normal University, Ministry of Educatio..."
4,10038845400,"[Nguyen, Dan Thanh T., Nguyen, Dan Thanh, Nguy...","[60023594, 60134796, 60012970, 60012970, 60012...","[Melbourne, Melbourne, Fulbourn, Bracknell, Br...","[Australia, Australia, United Kingdom, United ...",[Faculty of Pharmacy and Pharmaceutical Scienc...
...,...,...,...,...,...,...
907110,9943836000,"[Tsukamoto, Kyoko]",[60031720],[Fukuoka],[Japan],[Fukuoka University]
907111,9943884400,"[Zhang, Lian Zhen, Zhang, Lian Zhen, Zhang, Li...","[60082819, 118597277, 112096282, 100474394, 60...","[Shanghai, Shanghai, Shanghai, Shanghai, Shang...","[China, China, China, China, China]",[Shanghai Jiao Tong University School of Medic...
907112,9943985300,"[Toki, Haruhisa, Toki, Haruhisa]","[60029904, 60215776]","[Kamaishi, Tokyo]","[Japan, Japan]","[Marine Biotechnology Institute, Iwate, ENEOS ..."
907113,9944012700,"[Li, Sheng, Li, Sheng]","[60031069, 60031069]","[Leuven, 3000 Leuven]","[Belgium, Belgium]","[KU Leuven– University Hospital Leuven, KU Leu..."


## 发明人机构信息

In [28]:
# 发明人0的地址
df_match_result_with_address=df_match_result.merge(df_patstat_1[df_patstat_1['invt_seq_nr']==0][['appln_id','person_id','invt_seq_nr','psn_id','psn_name','person_address']],
                    left_on='appln_id_matching',
                    right_on='appln_id',
                      how='left'
                     )
df_match_result_with_address=df_match_result_with_address.rename({'appln_id':'invent_0_appln_id',
                                     'person_id':'invent_0_person_id',
                                     'invt_seq_nr':'invent_0_invt_seq_nr',
                                     'psn_id':'invent_0_psn_id',
                                     'psn_name':'invent_0_psn_name',
                                     'person_address':'invent_0_person_address'},axis=1)

#发明人本人的地址
df_match_result_with_address=df_match_result_with_address.merge(df_patstat_1[['appln_id','person_id','invt_seq_nr','psn_id','psn_name','person_address']],
                                                                left_on=['appln_id_matching','psn_id_matching'],
                                                                right_on=['appln_id','psn_id'],
                                                                  how='left'
                     )
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,invent_0_invt_seq_nr,invent_0_psn_id,invent_0_psn_name,invent_0_person_address,appln_id,person_id,invt_seq_nr,psn_id,psn_name,person_address
0,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,NaN,NaN,NaN,NaN,37450113,30581014,2.0,22531991,MORIMURA SHIGERU,None
1,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,NaN,NaN,NaN,NaN,37450113,30390915,3.0,29764767,SHIGEMATSU TORU,None
2,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,NaN,NaN,NaN,NaN,37450113,30396413,1.0,16955768,KIDA KENJI,None
3,4387,19338138100,29535451,Shen Jian,"Shen, You Cheng",479402407,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Elm flavor water caltrop vinegar brewing method,...,NaN,NaN,NaN,NaN,479402407,60001255,1.0,29535451,SHEN JIAN,None
4,4387,56687931500,34569678,Wang Ling,"Wang, Chin Kun",267426954,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Method for preparing pineapple peel residue fr...,...,0.0,11094413,GUANGDONG OCEAN UNIVERSITY,None,267426954,14865701,3.0,34569678,WANG LING,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17902107,2189,57244440000,19754519,"Liu, Kejing","Liu, Yang",413343242,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Hot air type scald instrument,...,0.0,29302937,SHANDONG UNIVERSITY,None,413343242,18708487,5.0,19754519,"LIU, KEJING",None
17902108,2189,57244440000,36171110,Yang Jianhua,"Liu, Yang",331512701,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Multi-functional electric heater,...,NaN,NaN,NaN,NaN,331512701,18723104,1.0,36171110,YANG JIANHUA,None
17902109,2189,57244440000,19738704,Liu Yiyang,"Liu, Yang",443602139,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Semiconductor refrigeration head hoop,...,NaN,NaN,NaN,NaN,443602139,40116858,1.0,19738704,LIU YIYANG,None
17902110,1192,56325243000,4759227,Cheng Lu,"Cheng, L. C.",520520428,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,Surgical dressing change frame,...,0.0,32455399,THE SECOND AFFILIATED HOSPITAL OF LUOHE MEDICA...,None,520520428,58882087,1.0,4759227,CHENG LU,None


In [29]:
df_match_result

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching
1378743,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.984326
1378743,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.978623
1378743,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,0.972987
8485011,4387,19338138100,29535451,Shen Jian,"Shen, You Cheng",479402407,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Elm flavor water caltrop vinegar brewing method,0.817428
8485011,4387,56687931500,34569678,Wang Ling,"Wang, Chin Kun",267426954,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Method for preparing pineapple peel residue fr...,0.817774
...,...,...,...,...,...,...,...,...,...,...,...
9285422,2189,57244440000,19754519,"Liu, Kejing","Liu, Yang",413343242,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Hot air type scald instrument,0.806795
9285422,2189,57244440000,36171110,Yang Jianhua,"Liu, Yang",331512701,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Multi-functional electric heater,0.818462
9285422,2189,57244440000,19738704,Liu Yiyang,"Liu, Yang",443602139,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Semiconductor refrigeration head hoop,0.850789
7820957,1192,56325243000,4759227,Cheng Lu,"Cheng, L. C.",520520428,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,Surgical dressing change frame,0.812837


## 增加学术机构信息

In [30]:
author_id_name={k:v for k,v in zip(df_match_result_author_id_aff_agg['author_id'],df_match_result_author_id_aff_agg['author_name'])}
author_id_afid={k:v for k,v in zip(df_match_result_author_id_aff_agg['author_id'],df_match_result_author_id_aff_agg['afid'])}
author_id_affiliation_city={k:v for k,v in zip(df_match_result_author_id_aff_agg['author_id'],df_match_result_author_id_aff_agg['affiliation_city'])}
author_id_affiliation_country={k:v for k,v in zip(df_match_result_author_id_aff_agg['author_id'],df_match_result_author_id_aff_agg['affiliation_country'])}
author_id_affilname={k:v for k,v in zip(df_match_result_author_id_aff_agg['author_id'],df_match_result_author_id_aff_agg['affilname'])}
author_id_afid

{'10038789900': ['60002836', '60001604', '60001604', '111155536'],
 '10038814800': ['60004321',
  '60172319',
  '60024705',
  '60003608',
  '60271156',
  '110245339',
  '60161458',
  '60016261',
  '60016261',
  '60016261',
  nan],
 '10038841100': ['60017161',
  '60017161',
  '60022195',
  '60022195',
  '60022195',
  '60022195',
  '60022195',
  '60022195',
  '60106371',
  '60028186',
  '60028186',
  '60028186',
  '60028186',
  '60070538'],
 '10038844100': ['60009559', '60001604', '60001604', '60019118', '60007566'],
 '10038845400': ['60023594', '60134796', '60012970', '60012970', '60012970'],
 '10038890300': ['60018029', '60018029'],
 '10038935700': ['60023256', '60023256', '60032256'],
 '10038945100': ['60009860',
  '60009860',
  '60009860',
  nan,
  '100355080',
  '60073655'],
 '10038986500': ['60022637'],
 '10039078400': ['60012981',
  '60012981',
  '60012981',
  '60012981',
  '60012981',
  '60012981',
  '113544051',
  '60000764',
  '60007363',
  '60007363',
  '60007363',
  '60007363

In [31]:
#学术作者的地址
df_match_result_with_address['author_name']=df_match_result_with_address['author_id'].map(author_id_name)
df_match_result_with_address['afid']=df_match_result_with_address['author_id'].map(author_id_afid)
df_match_result_with_address['affiliation_city']=df_match_result_with_address['author_id'].map(author_id_affiliation_city)
df_match_result_with_address['affiliation_country']=df_match_result_with_address['author_id'].map(author_id_affiliation_country)
df_match_result_with_address['affilname']=df_match_result_with_address['author_id'].map(author_id_affilname)
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname
0,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30581014,2.0,22531991,MORIMURA SHIGERU,None,"[Morimura, Shigeru, Morimura, S.]","[60021120, 60003498]","[Kumamoto, Kumamoto]","[Japan, Japan]","[Kumamoto University, Kumamoto University Scho..."
1,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30390915,3.0,29764767,SHIGEMATSU TORU,None,"[Shigematsu, Toru, Shigematsu, Toru, Shigemats...","[60024621, 60024621, 60021120, 60009613, 60103...","[Tsukuba, Tsukuba, Kumamoto, Niigata, Tokyo, T...","[Japan, Japan, Japan, Japan, Japan, Japan, Japan]",[National Institute of Advanced Industrial Sci...
2,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30396413,1.0,16955768,KIDA KENJI,None,"[Kida, Kenji, Kida, Kenji, Kida, Kenji]","[60016521, 60021120, 60021825]","[Chengdu, Kumamoto, Minato]","[China, Japan, Japan]","[Sichuan University, Kumamoto University, Toky..."
3,4387,19338138100,29535451,Shen Jian,"Shen, You Cheng",479402407,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Elm flavor water caltrop vinegar brewing method,...,60001255,1.0,29535451,SHEN JIAN,None,"[Shen, You Cheng, Shen, You Cheng]","[60027767, 60073390]","[Taichung, Taichung]","[Taiwan, Taiwan]","[Chung Shan Medical University, Chung Shan Med..."
4,4387,56687931500,34569678,Wang Ling,"Wang, Chin Kun",267426954,2-s2.0-85100102270,10.1016/j.smhs.2020.02.001,The ergogenic activity of cider vinegar: A ran...,Method for preparing pineapple peel residue fr...,...,14865701,3.0,34569678,WANG LING,None,"[Wang, Chin Kun]",[60027767],[Taichung],[Taiwan],[Chung Shan Medical University]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17902107,2189,57244440000,19754519,"Liu, Kejing","Liu, Yang",413343242,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Hot air type scald instrument,...,18708487,5.0,19754519,"LIU, KEJING",None,"[Liu, Yang]",[112918628],[Beijing],[China],[Beijing Institute of Satellite Environment En...
17902108,2189,57244440000,36171110,Yang Jianhua,"Liu, Yang",331512701,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Multi-functional electric heater,...,18723104,1.0,36171110,YANG JIANHUA,None,"[Liu, Yang]",[112918628],[Beijing],[China],[Beijing Institute of Satellite Environment En...
17902109,2189,57244440000,19738704,Liu Yiyang,"Liu, Yang",443602139,2-s2.0-85127692154,10.3760/cma.j.cn121430-20210330-00461,Design and application of a cooling device bas...,Semiconductor refrigeration head hoop,...,40116858,1.0,19738704,LIU YIYANG,None,"[Liu, Yang]",[112918628],[Beijing],[China],[Beijing Institute of Satellite Environment En...
17902110,1192,56325243000,4759227,Cheng Lu,"Cheng, L. C.",520520428,2-s2.0-0031606721,10.1097/00006534-199801000-00054,A simple tie-over dressing.,Surgical dressing change frame,...,58882087,1.0,4759227,CHENG LU,None,"[Cheng, L. C.]",[nan],[nan],[nan],[nan]


In [36]:
df_match_result_with_address.to_pickle('data/df_match_result_with_address.pickle')

# 开始匹配

In [10]:
df_match_result_with_address=pd.read_pickle('data/version/version_v1/df_match_result_with_address.pickle')
df_match_result_with_address=df_match_result_with_address[(df_match_result_with_address['affilname'].notnull())]
# df_match_result_with_address=df_match_result_with_address[['scopus_matching_name', 'patstat_matching_name','author_id', 'patstat_matching',
#                                                            'eid_matching', 'appln_id_matching',
#    'invent_0_person_address', 'person_address', 'affilname']]
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname
0,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30581014,2.0,22531991,MORIMURA SHIGERU,None,"[Morimura, Shigeru, Morimura, S.]","[60021120, 60003498]","[Kumamoto, Kumamoto]","[Japan, Japan]","[Kumamoto University, Kumamoto University Scho..."
1,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30390915,3.0,29764767,SHIGEMATSU TORU,None,"[Shigematsu, Toru, Shigematsu, Toru, Shigemats...","[60024621, 60024621, 60021120, 60009613, 60103...","[Tsukuba, Tsukuba, Kumamoto, Niigata, Tokyo, T...","[Japan, Japan, Japan, Japan, Japan, Japan, Japan]",[National Institute of Advanced Industrial Sci...
2,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,30396413,1.0,16955768,KIDA KENJI,None,"[Kida, Kenji, Kida, Kenji, Kida, Kenji]","[60016521, 60021120, 60021825]","[Chengdu, Kumamoto, Minato]","[China, Japan, Japan]","[Sichuan University, Kumamoto University, Toky..."
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co..."
4,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797731,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,79390590,0.0,24548116,"OTSUKI, JUNKO","1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 6...","[Otsuki, Junko, Otsuki, Junko, Otsuki, Junko, ...","[60006108, 100606334, 101983163, 60023147, 600...","[Ibaraki, Saitama, Kobe, Okayama, Okayama, Tok...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...","[National Institute of Biomedical Innovation, ..."
1797732,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,79390590,0.0,24548116,"OTSUKI, JUNKO","1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 6...","[Otsuki, Junko, Otsuki, Junko, Otsuki, Junko, ...","[60006108, 100606334, 101983163, 60023147, 600...","[Ibaraki, Saitama, Kobe, Okayama, Okayama, Tok...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...","[National Institute of Biomedical Innovation, ..."
1797733,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,79390590,0.0,24548116,"OTSUKI, JUNKO","1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 6...","[Otsuki, Junko

In [11]:
# df_match_result_with_address=df_match_result_with_address.explode(['author_name', 'afid', 'affiliation_city', 
#                                                                    'affiliation_country', 'affilname'])
# df_match_result_with_address=df_match_result_with_address.drop_duplicates()
# df_match_result_with_address

In [12]:
df_affilname_encoding=pd.DataFrame(df_match_result_with_address['affilname'].explode().astype(str).unique(),columns=['affilname'])
df_affilname_encoding['affilname_encoding']=list(model.encode(df_affilname_encoding['affilname'].astype(str).tolist(),convert_to_numpy=True,show_progress_bar=True))
df_affilname_encoding

Batches:   0%|          | 0/3845 [00:00<?, ?it/s]

,affilname,affilname_encoding
0,Kumamoto University,"[-0.029006729, 0.04481186, 0.043255184, -0.012..."
1,Kumamoto University School of Medicine,"[-0.016712537, 0.101421274, 0.015954433, -0.03..."
2,National Institute of Advanced Industrial Scie...,"[-0.1368436, 0.0103607, -0.06177567, 0.0070201..."
3,res,"[-0.11642302, 0.015433187, -0.046926267, 0.007..."
4,Niigata University of Pharmacy and Applied Lif...,"[-0.060993675, 0.032951467, -0.08504664, -0.00..."
...,...,...
123027,Practice for Human Genetics,"[-0.032726273, 0.047078215, -0.059404653, -0.0..."
123028,Embryonics R&D Center,"[-0.053457644, -0.075903095, -0.045653373, -0...."
123029,Universidade Estadual Júlio de Mesquita Filho ...,"[0.027224483, -0.026708422, 0.021325031, 0.016..."
123030,ARTechnic Co. Research and Development,"[-0.028816942, 0.06675292, -0.008376371, -0.03..."


In [13]:
dict_affilname_encoding={}
for k,v in zip(df_affilname_encoding['affilname'],df_affilname_encoding['affilname_encoding']):
    dict_affilname_encoding[k]=v

In [14]:
df_match_result_with_address.columns

Index(['labels', 'author_id', 'psn_id_matching', 'psn_id_matching_name',
       'scopus_matching_name', 'appln_id_matching', 'eid_matching',
       'doi_matching', 'title_scopus_matching', 'title_patstat_matching',
       'score_matching', 'invent_0_appln_id', 'invent_0_person_id',
       'invent_0_invt_seq_nr', 'invent_0_psn_id', 'invent_0_psn_name',
       'invent_0_person_address', 'appln_id', 'person_id', 'invt_seq_nr',
       'psn_id', 'psn_name', 'person_address', 'author_name', 'afid',
       'affiliation_city', 'affiliation_country', 'affilname'],
      dtype='object')

In [15]:
df_match_result_with_address['invent_0_psn_name_encoding']=list(model.encode(df_match_result_with_address['invent_0_psn_name'].astype(str).tolist(),convert_to_numpy=True,show_progress_bar=True))
df_match_result_with_address['invent_0_person_address_encoding']=list(model.encode(df_match_result_with_address['invent_0_person_address'].astype(str).tolist(),convert_to_numpy=True,show_progress_bar=True))
df_match_result_with_address['person_address_encoding']=list(model.encode(df_match_result_with_address['person_address'].astype(str).tolist(),convert_to_numpy=True,show_progress_bar=True))

Batches:   0%|          | 0/56180 [00:00<?, ?it/s]

Batches:   0%|          | 0/56180 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
any([x>6 for x in [1,2,3,4]])

False

In [17]:
df_match_result_with_address['affilname_encoding']=df_match_result_with_address['affilname'].apply(lambda x:get_dict(x,dict_affilname_encoding))

In [18]:
df_match_result_with_address['invent_0_psn_name_similarity']=[cos_sim(x,y)[0].numpy() for x,y in 
                                                              tqdm(zip(df_match_result_with_address['invent_0_psn_name_encoding'].tolist(),
                                                                       df_match_result_with_address['affilname_encoding'].tolist()))]
df_match_result_with_address['invent_0_person_address_similarity']=[cos_sim(x,y)[0].numpy() for x,y in 
                                                              tqdm(zip(df_match_result_with_address['invent_0_person_address_encoding'].tolist(),
                                                                       df_match_result_with_address['affilname_encoding'].tolist()))]
df_match_result_with_address['person_address_similarity']=[cos_sim(x,y)[0].numpy() for x,y in 
                                                              tqdm(zip(df_match_result_with_address['person_address_encoding'].tolist(),
                                                                       df_match_result_with_address['affilname_encoding'].tolist()))]

0it [00:00, ?it/s]/home/wangyuhang/.local/lib/python3.10/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  b = torch.tensor(b)
449616it [06:54, 1527.63it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

1797736it [28:35, 1048.10it/s]
1797736it [28:39, 1045.76it/s]
1797736it [28:24, 1054.55it/s]


In [19]:
# tem=df_match_result_with_address['invent_0_psn_name_similarity'].apply(lambda x:get_larger(x,0.7))
# tem

In [20]:
# len(tem[tem.apply(lambda x:len(x)>0)])

In [21]:
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,affiliation_city,affiliation_country,affilname,invent_0_psn_name_encoding,invent_0_person_address_encoding,person_address_encoding,affilname_encoding,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,"[Kumamoto, Kumamoto]","[Japan, Japan]","[Kumamoto University, Kumamoto University Scho...","[-0.0642979, -0.02824022, 0.045258436, -0.0351...","[-0.06429788, -0.028240232, 0.04525844, -0.035...","[0.0024044747, -0.04070058, -0.076731235, 0.01...","[[-0.029006729, 0.04481186, 0.043255184, -0.01...","[0.28137732, 0.22222334]","[0.28137738, 0.22222337]","[0.109581254, 0.057689425]"
1,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,"[Tsukuba, Tsukuba, Kumamoto, Niigata, Tokyo, T...","[Japan, Japan, Japan, Japan, Japan, Japan, Japan]",[National Institute of Advanced Industrial Sci...,"[-0.0642979, -0.02824022, 0.045258436, -0.0351...","[-0.0642979, -0.02824022, 0.045258436, -0.0351...","[0.002404461, -0.04070056, -0.07673123, 0.0199...","[[-0.1368436, 0.0103607, -0.06177567, 0.007020...","[0.14725946, 0.24151072, 0.28137735, 0.1459187...","[0.14725946, 0.24151072, 0.28137735, 0.1459187...","[0.08167138, 0.33279568, 0.10958125, 0.0548881..."
2,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,"[Chengdu, Kumamoto, Minato]","[China, Japan, Japan]","[Sichuan University, Kumamoto University, Toky...","[-0.0642979, -0.02824022, 0.045258436, -0.0351...","[-0.0642979, -0.02824022, 0.045258436, -0.0351...","[0.002404461, -0.04070056, -0.07673123, 0.0199...","[[-0.090305276, -0.019451186, -0.029335128, 0....","[0.3072466, 0.28137735, 0.19965106]","[0.3072466, 0.28137735, 0.19965106]","[0.12365728, 0.10958126, 0.051470563]"
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[-0.12813604, 0.085849494, -0.04739462, 0.0122...","[-0.020274319, -0.005732855, 0.014040356, 0.00...","[-0.0202743, -0.005732823, 0.014040347, 0.0098...","[[-0.13479866, 0.043942463, -0.029102277, 0.02...","[0.64877623, 0.6102642]","[0.37098807, 0.2993205]","[0.3709881, 0.29932055]"
4,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[-0.093497746, 0.03521996, -0.05208922, 0.0552...","[-0.09801738, 0.049089376, -0.039831735, -0.01...","[-0.0202743, -0.005732823, 0.014040347, 0.0098...","[[-0.13479866, 0.043942463, -0.029102277, 0.02...","[0.31797183, 0.2230064]","[0.27716115, 0.22692895]","[0.3709881, 0.29932055]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797731,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,"[Ibaraki, Saitama, Kobe, Okayama, Okayama, Tok...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...","[National Institute of Biomedical Innovation, ...","[-0.

In [23]:
df_match_result_with_address.to_pickle('data/version/version_v1/df_all.pickle')

# 检查数据

In [24]:
df_match_result_with_address=pd.read_pickle('data/version/version_v1/df_all.pickle')
df_match_result_with_address.drop(columns=['invent_0_psn_name_encoding','invent_0_person_address_encoding',
                                           'person_address_encoding','affilname_encoding'],inplace=True)
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,4387,7004114700,22531991,Morimura Shigeru,"Morimura, Shigeru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,MORIMURA SHIGERU,None,"[Morimura, Shigeru, Morimura, S.]","[60021120, 60003498]","[Kumamoto, Kumamoto]","[Japan, Japan]","[Kumamoto University, Kumamoto University Scho...","[0.28137732, 0.22222334]","[0.28137738, 0.22222337]","[0.109581254, 0.057689425]"
1,4387,7101829857,29764767,Shigematsu Toru,"Shigematsu, Toru",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,SHIGEMATSU TORU,None,"[Shigematsu, Toru, Shigematsu, Toru, Shigemats...","[60024621, 60024621, 60021120, 60009613, 60103...","[Tsukuba, Tsukuba, Kumamoto, Niigata, Tokyo, T...","[Japan, Japan, Japan, Japan, Japan, Japan, Japan]",[National Institute of Advanced Industrial Sci...,"[0.14725946, 0.24151072, 0.28137735, 0.1459187...","[0.14725946, 0.24151072, 0.28137735, 0.1459187...","[0.08167138, 0.33279568, 0.10958125, 0.0548881..."
2,4387,7102429514,16955768,Kida Kenji,"Kida, Kenji",37450113,2-s2.0-4544254605,10.1271/bbb.68.551,In vitro evaluation of physiological activity ...,FERMENTED VINEGAR AND METHOD FOR PRODUCING THE...,...,KIDA KENJI,None,"[Kida, Kenji, Kida, Kenji, Kida, Kenji]","[60016521, 60021120, 60021825]","[Chengdu, Kumamoto, Minato]","[China, Japan, Japan]","[Sichuan University, Kumamoto University, Toky...","[0.3072466, 0.28137735, 0.19965106]","[0.3072466, 0.28137735, 0.19965106]","[0.12365728, 0.10958126, 0.051470563]"
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[0.64877623, 0.6102642]","[0.37098807, 0.2993205]","[0.3709881, 0.29932055]"
4,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[0.31797183, 0.2230064]","[0.27716115, 0.22692895]","[0.3709881, 0.29932055]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797731,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,"OTSUKI, JUNKO","1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 6...","[Otsuki, Junko, Otsuki, Junko, Otsuki, Junko, ...","[60006108, 100606334, 101983163, 60023147, 600...","[Ibaraki, Saitama, Kobe, Okayama, Okayama, Tok...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...","[National Institute of Biomedical Innovation, ...","[0.34504053, 0.31466576, 0.15140289, 0.1843712...","[0.0948581, 0.23594339, 0.2374301, 0.1098783, ...","[0.018318456, 0.14769404, 0.16918622, 0.030594..."
1797732,3376,56256023400,24548116,"Otsuki, Junko","Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,"OTSUKI, JUNKO","1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 6...","[Otsu

In [25]:
df_match_result_with_address=df_match_result_with_address[
    (df_match_result_with_address['invent_0_psn_name_similarity'].apply(lambda x:compare_larger(x,0.6)))|
 (df_match_result_with_address['invent_0_person_address_similarity'].apply(lambda x:compare_larger(x,0.6)))|
    (df_match_result_with_address['person_address_similarity'].apply(lambda x:compare_larger(x,0.6)))
                                                         ]
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[0.64877623, 0.6102642]","[0.37098807, 0.2993205]","[0.3709881, 0.29932055]"
5,651,57632820300,36072517,"Yamamoto, Itsuo","Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"YAMAMOTO, ITSUO","Yao-shi, Osaka","[Yamamoto, Itsuo, Yamamoto, Itsuo]","[100613492, 101815157]","[Osaka, Osaka]","[Japan, Japan]","[Yamamoto Vinita Co. Ltd., Yamamoto Vinyter Co...","[0.64877623, 0.6102642]","[0.37098807, 0.2993205]","[0.3709881, 0.29932055]"
7,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"[Robins, H. I., Robins, H. I., Robins, Henry I...","[60016534, 60032179, 60011940, 60014455, 60025...","[Madison, Madison, Madison, Madison, Madison, ...","[United States, United States, United States, ...","[UW Health, University of Wisconsin-Madison, U...","[0.31008786, 0.36234215, 0.6613666, 0.28220242...","[0.20971116, 0.35216755, 0.13287245, 0.340376,...","[0.14939798, 0.70395213, 0.3404087, 0.4607318,..."
8,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"[Robins, H. I., Robins, H. I., Robins, Henry I...","[60016534, 60032179, 60011940, 60014455, 60025...","[Madison, Madison, Madison, Madison, Madison, ...","[United States, United States, United States, ...","[UW Health, University of Wisconsin-Madison, U...","[0.31008786, 0.36234215, 0.6613666, 0.28220242...","[0.18224007, 0.15777393, 0.10194931, 0.2239749...","[0.18224007, 0.15777393, 0.10194931, 0.2239749..."
9,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"[Robins, H. I., Robins, H. I., Robins, Henry I...","[60016534, 60032179, 60011940, 60014455, 60025...","[Madison, Madison, Madison, Madison, Madison, ...","[United States, United States, United States, ...","[UW Health, University of Wisconsin-Madison, U...","[0.31008786, 0.36234215, 0.6613666, 0.28220242...","[0.20971116, 0.35216755, 0.13287245, 0.340376,...","[0.14939798, 0.70395213, 0.3404087, 0.4607318,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797723,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85021428936,10.1002/rmb2.12038,Retrospective comparative study of the factors...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","[Otsuki, Junko, Otsuki, Junko, Otsuki, Junko, ...","[60006108, 100606334, 101983163, 60023147, 600...","[Ibaraki, Saitama, Kobe, Okayama, Okayama, Tok...","[Japan, Japan, Japan, Japan, Japan, Japan, Jap...","[National Institute of Biomedical Innovation, ...","[0.12022062, 0.26942015, 0.27215108, 0.2058212...","[0.030181395, 0.11813758, 0.25329792, 0.113185..."

In [26]:
df_match_result_with_address=df_match_result_with_address.explode(['author_name','afid',
                                                                   'affiliation_city',
                                                                   'affiliation_country',
                                                                   'affilname',
                                                                    'invent_0_psn_name_similarity',
                                                                   'invent_0_person_address_similarity',
                                                                   'person_address_similarity'])
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
3,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",101815157,Osaka,Japan,Yamamoto Vinyter Co. Ltd.,0.610264,0.29932,0.299321
5,651,57632820300,36072517,"Yamamoto, Itsuo","Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"YAMAMOTO, ITSUO","Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
5,651,57632820300,36072517,"Yamamoto, Itsuo","Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"YAMAMOTO, ITSUO","Yao-shi, Osaka","Yamamoto, Itsuo",101815157,Osaka,Japan,Yamamoto Vinyter Co. Ltd.,0.610264,0.29932,0.299321
7,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.209711,0.149398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797730,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, Junko",123395422,"Kobe, Okayama",Japan,Hanabusa Women's Clinic,0.088545,0.169186,0.760937
1797730,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, J.",100477586,Misato,Japan,Nagai Clinic,0.162326,0.147694,0.50696
1797730,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, J.",60018642,Tokyo,Japan,Toho University,0.244798,0.202378,0.211407
1797730,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85081307887,10.1002/rmb2.12322,Predictive factors influencing pregnancy rate ...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, Junko",100866168,Hyogo Prefecture,Japan,Hanabusa Women's Clinic,0.088545,0.169186,0.760937


In [27]:
df_match_result_with_address=df_match_result_with_address.drop_duplicates(['appln_id_matching','eid_matching'])
df_match_result_with_address=df_match_result_with_address[df_match_result_with_address['affilname'].notnull()]
df_match_result_with_address=df_match_result_with_address.reset_index(drop=True)
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.209711,0.149398
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.18224,0.18224
3,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.209711,0.149398
4,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.18224,0.18224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717255,3376,56276227600,28263656,Samir Hamamah,"Hamamah, Samir",520316723,2-s2.0-85061146370,10.1016/j.gofs.2019.01.011,Comparison of birth weights of children born a...,METHODS FOR DETERMINING QUALITY OF EMBRYO,...,SAMIR HAMAMAH,None,"Hamamah, S.",60027381,Tours,France,Hopital Bretonneau,0.219571,0.140577,0.140577
717256,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,...,"HOUGHTON, FRANCHESCA, D.",Southampton,"Houghton, F. D.",60016418,York,United Kingdom,University of York,0.285791,0.136658,0.321236
717257,3376,54996999300,30182386,"Silver, David H.","Silver, David H.",564766225,2-s2.0-85139535700,10.1093/humrep/deac171,Embryologist agreement when assessing blastocy...,PREDICTING EMBRYO IMPLANTATION PROBABILITY,...,"SILVER, DAVID H.",None,"Silver, David H.",60022403,Haifa,Israel,L'Institut Agro Rennes-Angers,0.223637,0.098552,0.098552
717258,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85021428936,10.1002/rmb2.12038,Retrospective comparative study of the factors...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, Junko",60006108,Ibaraki,Japan,National Institute of Biomedical Innovation,0.345041,0.094858,0.218905


In [ ]:
print('专利论文对',len(df_match_result_with_address))
print('论文数量',len(df_match_result_with_address['eid_matching'].unique()))
print('专利数量',len(df_match_result_with_address['appln_id_matching'].unique()))
print('作者数量',len(df_match_result_with_address['author_id'].unique()))

In [34]:
df_match_result_with_address.to_pickle('data/version/version_v1/Paper_Patent_Pair_v1.pickle')
df_match_result_with_address.to_csv('data/version/version_v1/Paper_Patent_Pair_v1.csv',encoding='utf-8-sig',index=None)
df_match_result_with_address[:10000].to_csv('data/version/version_v1/Paper_Patent_Pair_v1_sample.csv',encoding='utf-8-sig',index=None)

In [2]:
df_match_result_with_address=pd.read_pickle('data/version/version_v1/Paper_Patent_Pair_v1.pickle')
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949
3,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952
4,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709951,3376,56276227600,28263656,Samir Hamamah,"Hamamah, Samir",520316723,2-s2.0-85061146370,10.1016/j.gofs.2019.01.011,Comparison of birth weights of children born a...,METHODS FOR DETERMINING QUALITY OF EMBRYO,...,SAMIR HAMAMAH,None,"Hamamah, S.",60108488,Montpellier,France,Université de Montpellier,0.79196,0.170101,0.170101
709952,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,...,"HOUGHTON, FRANCHESCA, D.",Southampton,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,0.064718,0.123707,0.728192
709953,3376,54996999300,30182386,"Silver, David H.","Silver, David H.",564766225,2-s2.0-85139535700,10.1093/humrep/deac171,Embryologist agreement when assessing blastocy...,PREDICTING EMBRYO IMPLANTATION PROBABILITY,...,"SILVER, DAVID H.",None,"Silver, David H.",128683168,Haifa,Israel,Embryonics R&D Center,0.721141,0.068483,0.068483
709954,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85021428936,10.1002/rmb2.12038,Retrospective comparative study of the factors...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, Junko",101983163,Kobe,Japan,Hanabusa Women's Clinic,0.151403,0.23743,0.760937


In [159]:
df=pd.read_pickle('data/version/version_v1/Paper_Patent_Pair_v1.pickle')
df

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching,invent_0_appln_id,invent_0_person_id,invent_0_invt_seq_nr,invent_0_psn_id,invent_0_psn_name,invent_0_person_address,appln_id,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,0.981034,52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka",52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949
3,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952
4,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709951,3376,56276227600,28263656,Samir Hamamah,"Hamamah, Samir",520316723,2-s2.0-85061146370,10.1016/j.gofs.2019.01.011,Comparison of birth weights of children born a...,METHODS FOR DETERMINING QUALITY OF EMBRYO,0.954320,520316723,64964638,0.0,33577244,UNIV MONTPELLIER,None,520316723,62535779,1.0,28263656,SAMIR HAMAMAH,None,"Hamamah, S.",60108488,Montpellier,France,Université de Montpellier,0.79196,0.170101,0.170101
709952,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,0.980987,267368610,11131393,0.0,23886831,NOVOCELLUS,None,267368610,11131395,2.0,13258239,"HOUGHTON, FRANCHESCA, D.",Southampton,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,0.064718,0.123707,0.728192
709953,3376,54996999300,30182386,"Silver, David H.","Silver, David H.",564766225,2-s2.0-85139535700,10.1093/humrep/deac171,Embryologist agreement when assessing blastocy...,PREDICTING EMBRYO IMPLANTATION PROBA

In [180]:
len(df['author_id'].unique())

83315

# 回应comment 专利家族

In [12]:
query = ''' SELECT t."appln_id", t."appln_kind", t."appln_auth", t."appln_nr_original", t."appln_nr_original", t."docdb_family_id" FROM "biotechnology_basic_info" t '''
df_basic_info=pd.read_sql(query, connection)
df_basic_info['appln_id']=df_basic_info['appln_id'].astype(str)
df_basic_info

/tmp/ipykernel_67607/882448606.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_basic_info=pd.read_sql(query, connection)


,appln_id,appln_kind,appln_auth,appln_nr_original,appln_nr_original,docdb_family_id
0,72,A,EP,04799975,04799975,36676151
1,79,A,EP,05023512,05023512,35677469
2,88,A,EP,05715031,05715031,37818140
3,116,A,EP,06706886,06706886,34933703
4,119,A,EP,06753961,06753961,37000110
...,...,...,...,...,...,...
10676891,575014548,U,CN,202121421775,202121421775,82121976
10676892,575014578,U,CN,202120539293,202120539293,75539902
10676893,575014590,U,CN,202121439309,202121439309,82121995
10676894,575014598,U,CN,202122451292,202122451292,82121999


In [13]:
len(df_match_result_with_address['author_id'].unique())

67777

In [14]:
df_match_result_with_address=pd.read_pickle('data/version/version_v1/Paper_Patent_Pair_v1.pickle')
df_match_result_with_address=df_match_result_with_address.merge(df_basic_info[['appln_id','docdb_family_id']],how='left',left_on='appln_id',right_on='appln_id')
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching,invent_0_appln_id,invent_0_person_id,invent_0_invt_seq_nr,invent_0_psn_id,invent_0_psn_name,invent_0_person_address,appln_id,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity,docdb_family_id
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,0.981034,52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka",52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988,16506615
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202
3,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202
4,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709951,3376,56276227600,28263656,Samir Hamamah,"Hamamah, Samir",520316723,2-s2.0-85061146370,10.1016/j.gofs.2019.01.011,Comparison of birth weights of children born a...,METHODS FOR DETERMINING QUALITY OF EMBRYO,0.954320,520316723,64964638,0.0,33577244,UNIV MONTPELLIER,None,520316723,62535779,1.0,28263656,SAMIR HAMAMAH,None,"Hamamah, S.",60108488,Montpellier,France,Université de Montpellier,0.79196,0.170101,0.170101,48698981
709952,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,0.980987,267368610,11131393,0.0,23886831,NOVOCELLUS,None,267368610,11131395,2.0,13258239,"HOUGHTON, FRANCHESCA, D.",Southampton,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,0.064718,0.123707,0.728192,36061050
709953,3376,54996999300,30182386,"Silver, David H.","Silver, David H.",564766225,2-s2.0-85139535700,10.1093/humrep/deac171,Emb

In [15]:
query = ''' SELECT t.* FROM "biotechnology_author" t '''
df_biotechnology_author=pd.read_sql(query, connection)
# df_scopus=pd.read_pickle('data/df_scopus.pickle')
df_biotechnology_author['appln_id']=df_biotechnology_author['appln_id'].astype(str)
df_biotechnology_author


/tmp/ipykernel_67607/391992716.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_biotechnology_author=pd.read_sql(query, connection)


,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address
0,159987,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
1,300020,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
2,337604,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
3,358478,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
4,15740590,16.0,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891.0,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
...,...,...,...,...,...,...,...
40251194,574737563,85433144.0,1.0,"KERR, Amy",185433144.0,UNKNOWN,None
40251195,418030930,85440307.0,0.0,ORGANIC MEDICAL VENTURES L.L.C.,185440307.0,UNKNOWN,"3209 NORTH FLOOD STREET \n SUITE 7 \n NORMAN, ..."
40251196,568757381,85452070.0,0.0,SEMICONDUCTOR ENERGY RESEARCH INSTITUTE FOR JO...,185452070.0,UNKNOWN,None
40251197,557660589,85430910.0,2.0,"DOBSON, Justin",185430910.0,UNKNOWN,None


In [16]:
df_biotechnology_author=df_biotechnology_author[df_biotechnology_author['invt_seq_nr']>0]
df_biotechnology_author

,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address
158,17028660,19.0,4.0,"WINTER, GREGORY PAUL",35376353.0,None,"64 Cavendish Avenue,Cambridge CB1 4UT"
188,442251195,47.0,1.0,"SUNDHOLM, GOERAN",31485533.0,INDIVIDUAL,"Ilmari Kiannon Kuja 3,04310 Tuusula"
189,484535604,47.0,2.0,"SUNDHOLM, GOERAN",31485533.0,INDIVIDUAL,"Ilmari Kiannon Kuja 3,04310 Tuusula"
753,169551,274.0,7.0,"WACHENDORFF-NEUMANN, ULRIKE",34377885.0,None,"Oberer Markenweg 85,56566 Neuwied"
754,15793582,274.0,8.0,"WACHENDORFF-NEUMANN, ULRIKE",34377885.0,None,"Oberer Markenweg 85,56566 Neuwied"
...,...,...,...,...,...,...,...
40251192,571323564,85440714.0,1.0,Richard Gagliardo,185440714.0,UNKNOWN,None
40251193,569882343,85430869.0,1.0,DO CUONG,185430869.0,UNKNOWN,None
40251194,574737563,85433144.0,1.0,"KERR, Amy",185433144.0,UNKNOWN,None
40251197,557660589,85430910.0,2.0,"DOBSON, Justin",185430910.0,UNKNOWN,None


In [17]:
# df_biotechnology_author统计了每个person_id发的专利appln_id，我们补充了每个appln_id的专利家族号，也就是获得了每个person_id发表的专利家族号
df_biotechnology_author=df_biotechnology_author.merge(df_basic_info[['appln_id','docdb_family_id']])
df_biotechnology_author

,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address,docdb_family_id
0,17028660,19.0,4.0,"WINTER, GREGORY PAUL",35376353.0,None,"64 Cavendish Avenue,Cambridge CB1 4UT",10722895
1,17028660,3304510.0,3.0,"STEVENSON, FREDA KATHERINE",31110540.0,None,"9 Meadowhead Road,Bassett, Southampton SO1 7AD",10722895
2,17028660,3304409.0,1.0,"HAWKINS, ROBERT EDWARD",12163584.0,None,"6 The Lawns,Cambridge CB3 0RU",10722895
3,17028660,3304509.0,2.0,"RUSSELL, STEPHEN JAMES",27916841.0,None,"10 Cortyards, Little Shelford,Cambridge CB2 5ER",10722895
4,442251195,47.0,1.0,"SUNDHOLM, GOERAN",31485533.0,INDIVIDUAL,"Ilmari Kiannon Kuja 3,04310 Tuusula",52808367
...,...,...,...,...,...,...,...,...
29848419,567496993,85655446.0,1.0,KITANO KEI,185655446.0,UNKNOWN,None,80324074
29848420,513653332,85627247.0,1.0,FAWARSY SUMMER,185627247.0,UNKNOWN,MAGHER ZIP CODE P.O.B. 517 20128 IL,60912469
29848421,575103768,85518482.0,1.0,"ZHANG, Jiangang",185518482.0,UNKNOWN,None,82158788
29848422,574173716,85534423.0,1.0,"Kervalishvili, Givi",185534423.0,UNKNOWN,None,81927166


In [18]:
df_biotechnology_author['person_id']=df_biotechnology_author['person_id'].astype(int).astype(str)

In [19]:
# 每个person_id和psn_id是对应关联的，所以就获得了每一个psn_id的专利家族号
df_tem=df_biotechnology_author.groupby(['psn_id','psn_name','docdb_family_id']).agg(
                                                            appln_ids=('appln_id',list),
                                                            person_ids=('person_id',list),
                                                            invt_seq_nrs=('invt_seq_nr',list),
                                                            psn_names=('psn_name',list),
                                                                   ).reset_index()
df_tem['psn_id']=df_tem['psn_id'].astype(int).astype(str)
df_tem

,psn_id,psn_name,docdb_family_id,appln_ids,person_ids,invt_seq_nrs,psn_names
0,5,-,6461752,[7702716],[14730952],[5.0],[-]
1,5,-,6529939,[7624770],[14730952],[8.0],[-]
2,5,-,7834914,[7671267],[14730952],[5.0],[-]
3,5,-,7894006,[6623902],[14730952],[4.0],[-]
4,5,-,8177040,[6904445],[14730952],[4.0],[-]
...,...,...,...,...,...,...,...
19189200,185724143,"ZUR, Amir",76553779,[552871457],[85724143],[1.0],"[ZUR, Amir]"
19189201,185724144,"ZWICKER, Jeffery",78844846,[562511787],[85724144],[3.0],"[ZWICKER, Jeffery]"
19189202,185724169,"BEUKES, Natasha",77655585,[556747196],[85724169],[2.0],"[BEUKES, Natasha]"
19189203,185724183,"GWANZURA, Zvikomborero Takunda",80490707,[568211406],[85724183],[1.0],"[GWANZURA, Zvikomborero Takunda]"


In [20]:
df_tem1=df_tem[df_tem['appln_ids'].apply(lambda x:len(x)>1)]
df_tem1

,psn_id,psn_name,docdb_family_id,appln_ids,person_ids,invt_seq_nrs,psn_names
15,5,-,23145037,"[6908027, 6908027]","[14730952, 14730952]","[3.0, 11.0]","[-, -]"
23,5,-,43627099,"[411142672, 411142672]","[69151145, 68858479]","[4.0, 2.0]","[-, -]"
24,5,-,50279595,"[416219138, 416219138, 416219138, 416219138]","[67949980, 58089806, 68208103, 58090647]","[1.0, 5.0, 2.0, 4.0]","[-, -, -, -]"
35,107,"' T HOEN, BARTHOLOMEUS CORNELIS",56561371,"[459171923, 459031609]","[56795824, 55393187]","[2.0, 2.0]","[' T HOEN, BARTHOLOMEUS CORNELIS, ' T HOEN, BA..."
59,315,( DICKON) ALLEY M.R.K.,55409879,"[497866059, 497866075]","[53695283, 53695283]","[4.0, 3.0]","[( DICKON) ALLEY M.R.K., ( DICKON) ALLEY M.R.K.]"
...,...,...,...,...,...,...,...
19189033,185723513,"SHIVASWAMY, Sushma",81077418,"[570574576, 570823988]","[85723513, 85723513]","[2.0, 2.0]","[SHIVASWAMY, Sushma, SHIVASWAMY, Sushma]"
19189034,185723513,"SHIVASWAMY, Sushma",81077421,"[570574687, 570824056]","[85723513, 85723513]","[2.0, 2.0]","[SHIVASWAMY, Sushma, SHIVASWAMY, Sushma]"
19189041,185723523,"SIDDIQUI, MUTEEB",73457393,"[563053395, 570347037]","[85723523, 85723523]","[2.0, 2.0]","[SIDDIQUI, MUTEEB, SIDDIQUI, MUTEEB]"
19189117,185723865,"VAIL, Jake",79687603,"[565250930, 565251007]","[85723865, 85723865]","[5.0, 5.0]","[VAIL, Jake, VAIL, Jake]"


In [21]:
# df_match_result_with_address['psn_id']=df_match_result_with_address['psn_id'].astype(float)

In [22]:
df_match_result_with_address.columns

Index(['labels', 'author_id', 'psn_id_matching', 'psn_id_matching_name',
       'scopus_matching_name', 'appln_id_matching', 'eid_matching',
       'doi_matching', 'title_scopus_matching', 'title_patstat_matching',
       'score_matching', 'invent_0_appln_id', 'invent_0_person_id',
       'invent_0_invt_seq_nr', 'invent_0_psn_id', 'invent_0_psn_name',
       'invent_0_person_address', 'appln_id', 'person_id', 'invt_seq_nr',
       'psn_id', 'psn_name', 'person_address', 'author_name', 'afid',
       'affiliation_city', 'affiliation_country', 'affilname',
       'invent_0_psn_name_similarity', 'invent_0_person_address_similarity',
       'person_address_similarity', 'docdb_family_id'],
      dtype='object')

In [23]:
df_match_result_with_address2=df_match_result_with_address.merge(df_tem[['psn_id','docdb_family_id','appln_ids','person_ids']],
                                                                how='left',left_on=['psn_id','docdb_family_id'],right_on=['psn_id','docdb_family_id'])
df_match_result_with_address2

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching,invent_0_appln_id,invent_0_person_id,invent_0_invt_seq_nr,invent_0_psn_id,invent_0_psn_name,invent_0_person_address,appln_id,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity,docdb_family_id,appln_ids,person_ids
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,0.981034,52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka",52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988,16506615,NaN,NaN
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202,"[46482242, 4509247]","[5762813, 16317553]"
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202,"[47242438, 1290643, 17077175]","[37692963, 13455904, 3398868]"
3,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202,"[46482242, 4509247]","[5762813, 16317553]"
4,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202,"[47242438, 1290643, 17077175]","[37692963, 13455904, 3398868]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709990,3376,56276227600,28263656,Samir Hamamah,"Hamamah, Samir",520316723,2-s2.0-85061146370,10.1016/j.gofs.2019.01.011,Comparison of birth weights of children born a...,METHODS FOR DETERMINING QUALITY OF EMBRYO,0.954320,520316723,64964638,0.0,33577244,UNIV MONTPELLIER,None,520316723,62535779,1.0,28263656,SAMIR HAMAMAH,None,"Hamamah, S.",60108488,Montpellier,France,Université de Montpellier,0.79196,0.170101,0.170101,48698981,"[520316723, 567010260]","[62535779, 62535779]"
709991,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,0.980987,267368610,11131393,0.0,23886831,NOVOCELLUS,None,267368610,11

In [24]:
df_match_result_with_address3=df_match_result_with_address2.explode('person_ids')
df_match_result_with_address3

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,score_matching,invent_0_appln_id,invent_0_person_id,invent_0_invt_seq_nr,invent_0_psn_id,invent_0_psn_name,invent_0_person_address,appln_id,person_id,invt_seq_nr,psn_id,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity,docdb_family_id,appln_ids,person_ids
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,0.981034,52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka",52568004,9384898,0.0,36079383,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988,16506615,NaN,NaN
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202,"[46482242, 4509247]",5762813
1,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,1.000000,46482242,5762811,0.0,4033241,CANCER RESEARCH INSTITUTE,New York,46482242,5762813,2.0,27394585,"ROBINS, H. IAN",Madison,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,0.362342,0.352168,0.703952,21987202,"[46482242, 4509247]",16317553
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202,"[47242438, 1290643, 17077175]",37692963
2,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,0.990813,47242438,13455903,0.0,4033241,CANCER RESEARCH INSTITUTE,None,47242438,37692963,1.0,27394586,"ROBINS, H., IAN",None,"Robins, Henry Ian",60011940,Madison,United States,University of Wisconsin Carbone Cancer Center,0.661367,0.101949,0.101949,21987202,"[47242438, 1290643, 17077175]",13455904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709991,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,0.980987,267368610,11131393,0.0,23886831,NOVOCELLUS,None,267368610,11131395,2.0,13258239,"HOUGHTON, FRANCHESCA, D.",Southampton,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,0.064718,0.123707,0.728192,36061050,"[267368610, 23700632, 231028]",29246631
709991,3376,7005625766,13258239,"Houghton, Franchesca, D.","Houghton, Franchesca D.",267368610,2-s2.0-85143180655,10.1093/humrep/deac233,Incubator type affects human blastocyst format...,Method of assessing the viability of thawed de...,0.980987,267368610,11131393,0.0,23886831,NOVOCELLUS,None,267368610,11131395,2.0,13258239,"HOUGHTO

In [25]:
df_match_result_with_address3[df_match_result_with_address3['author_id']=='56256023400']['person_address'].unique()

array(["c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-cho,Chuo-ku,Kobe-shi, Hyogo 650-0021"],
      dtype=object)

In [25]:
df_match_result_with_address3['person_ids']=df_match_result_with_address3['person_ids'].fillna(0).astype(int).astype(str)

In [26]:
len(df_match_result_with_address3['author_id'].unique())

83315

In [27]:
len(df_match_result_with_address3['person_ids'].unique())

146369

In [28]:
len(df_match_result_with_address3['person_id'].unique())

91804

In [29]:
len(set(df_match_result_with_address3['person_id'].tolist()+df_match_result_with_address3['person_ids'].tolist()))

146570

In [30]:
dict_personid1_authorid={k:v for k,v in zip(df_match_result_with_address3['person_id'],df_match_result_with_address3['author_id'])}
dict_personid2_authorid={k:v for k,v in zip(df_match_result_with_address3['person_ids'],df_match_result_with_address3['author_id'])}

In [31]:
dict_personid_authorid=dict(dict_personid1_authorid,**dict_personid2_authorid)

In [32]:
dict_personid_authorid['9384898']

'57632820300'

In [33]:
df_match_result_with_address.columns

Index(['labels', 'author_id', 'psn_id_matching', 'psn_id_matching_name',
       'scopus_matching_name', 'appln_id_matching', 'eid_matching',
       'doi_matching', 'title_scopus_matching', 'title_patstat_matching',
       'score_matching', 'invent_0_appln_id', 'invent_0_person_id',
       'invent_0_invt_seq_nr', 'invent_0_psn_id', 'invent_0_psn_name',
       'invent_0_person_address', 'appln_id', 'person_id', 'invt_seq_nr',
       'psn_id', 'psn_name', 'person_address', 'author_name', 'afid',
       'affiliation_city', 'affiliation_country', 'affilname',
       'invent_0_psn_name_similarity', 'invent_0_person_address_similarity',
       'person_address_similarity', 'docdb_family_id'],
      dtype='object')

In [34]:
df_biotechnology_author.columns

Index(['appln_id', 'person_id', 'invt_seq_nr', 'psn_name', 'psn_id',
       'psn_sector', 'person_address', 'docdb_family_id'],
      dtype='object')

In [35]:
# !mkdir data/version/version_v2

In [36]:
df_match_result_with_address_v2=pd.DataFrame.from_dict(dict_personid_authorid,orient='index').reset_index()
df_match_result_with_address_v2.columns=['person_id','author_id',]
df_match_result_with_address_v2=df_match_result_with_address_v2.merge(df_match_result_with_address[['author_id', 'author_name', 'afid',
       'affiliation_city', 'affiliation_country', 'affilname']].drop_duplicates('author_id'),how='left',left_on='author_id',right_on='author_id')
df_match_result_with_address_v2=df_match_result_with_address_v2.merge(df_biotechnology_author[['person_id', 'psn_name', 'person_address']
                                                     ].drop_duplicates('person_id'),how='left',left_on='person_id',right_on='person_id')

df_match_result_with_address_v2=df_match_result_with_address_v2[df_match_result_with_address_v2['person_id']!='0']

df_match_result_with_address_v2=df_match_result_with_address_v2[~df_match_result_with_address_v2['psn_name'].isnull()]
df_match_result_with_address_v2.to_pickle('data/version/version_v2/Paper_Patent_Pair_v2.pickle')

df_match_result_with_address_v2

,person_id,author_id,author_name,afid,affiliation_city,affiliation_country,affilname,psn_name,person_address
1,5762813,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison
2,37692963,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None
3,5432419,7101821912,"Ash, Stephen R.",113166379,Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette
4,71519745,35248974600,"Saito, Hajime",60027841,Akita,Japan,Akita University,SAITO HAJIME,None
5,32196003,7003592507,"Mitobe, Kazutaka",60027841,Akita,Japan,Akita University,MITOBE KAZUTAKA,None
...,...,...,...,...,...,...,...,...,...
146565,12621324,7005818600,"Grifo, Jamie A.",60021784,New York,United States,New York University,"GRIFO, JAMIE",None
146566,29246630,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.",None
146567,369337,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ..."
146568,29246631,7005625766,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None


# 分国家检查

In [37]:
df_match_result_with_address_v2=pd.read_pickle('data/version/version_v2/Paper_Patent_Pair_v2.pickle')
df_match_result_with_address_v2.to_csv('data/version/version_v2/Paper_Patent_Pair_v2.csv',encoding='utf-8-sig',index=None)

df_match_result_with_address_v2

,person_id,author_id,author_name,afid,affiliation_city,affiliation_country,affilname,psn_name,person_address
1,5762813,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison
2,37692963,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None
3,5432419,7101821912,"Ash, Stephen R.",113166379,Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette
4,71519745,35248974600,"Saito, Hajime",60027841,Akita,Japan,Akita University,SAITO HAJIME,None
5,32196003,7003592507,"Mitobe, Kazutaka",60027841,Akita,Japan,Akita University,MITOBE KAZUTAKA,None
...,...,...,...,...,...,...,...,...,...
146565,12621324,7005818600,"Grifo, Jamie A.",60021784,New York,United States,New York University,"GRIFO, JAMIE",None
146566,29246630,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.",None
146567,369337,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ..."
146568,29246631,7005625766,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None


In [183]:
len(df_match_result_with_address_v2['person_id'].unique())

146397

In [38]:


query = ''' SELECT t.* FROM "Paper_Patent_Pair_v2_person_tls206" t '''
Paper_Patent_Pair_v2_person_tls206=pd.read_sql(query, connection)
# df_scopus=pd.read_pickle('data/df_scopus.pickle')
Paper_Patent_Pair_v2_person_tls206['person_id']=Paper_Patent_Pair_v2_person_tls206['person_id'].astype(int).astype(str)
Paper_Patent_Pair_v2_person_tls206


/tmp/ipykernel_16214/2964252143.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Paper_Patent_Pair_v2_person_tls206=pd.read_sql(query, connection)


,person_id,person_name,person_name_orig_lg,person_address,person_ctry_code,psn_id,psn_name,psn_sector
0,44830521,Zhang Yuan-Peng,Zhang Yuan-Peng,"Suite 105, 40 East 11th Avenue,Vancouver, Brit...",CA,37084028,ZHANG YUAN-PENG,None
1,3635784,"ZHANG, Yuan-Peng","ZHANG, Yuan-Peng","Suite 105 40 East 11th Avenue,Vancouver, Briti...",CA,37104984,"ZHANG, YUAN-PENG",None
2,40491450,REIMER DOROTHY L,REIMER DOROTHY L,None,,26848285,REIMER DOROTHY L,None
3,16772483,"HARVIE, PIERROT","HARVIE, PIERROT",None,US,12066168,"HARVIE, PIERROT",INDIVIDUAL
4,32385379,"AKITA, HIDETAKA","AKITA, HIDETAKA",None,JP,506466,AKITA HIDETAKA,INDIVIDUAL
...,...,...,...,...,...,...,...,...
146392,11373862,"Minucci, Saverio","Minucci, Saverio",Noverasco Di Opera,IT,22126273,"MINUCCI, SAVERIO",INDIVIDUAL
146393,40200931,"MINUCCI, SAVERIO","MINUCCI, SAVERIO",None,,22126273,"MINUCCI, SAVERIO",INDIVIDUAL
146394,14103944,SAVERIO MINUCCI,SAVERIO MINUCCI,None,,28513761,SAVERIO MINUCCI,INDIVIDUAL
146395,37144233,"SINCLAIR, DAVID, A.","SINCLAIR, DAVID, A.",None,US,30247909,"SINCLAIR, DAVID, A.",INDIVIDUAL


In [39]:
df_match_result_with_address_v2=df_match_result_with_address_v2.merge(Paper_Patent_Pair_v2_person_tls206[['person_id','person_name','person_name_orig_lg','person_address','person_ctry_code']])
df_match_result_with_address_v2

,person_id,author_id,author_name,afid,affiliation_city,affiliation_country,affilname,psn_name,person_address,person_name,person_name_orig_lg,person_ctry_code
0,5762813,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison,"Robins, H. Ian","Robins, H. Ian",US
1,37692963,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None,"ROBINS, H., IAN","ROBINS, H., IAN",
2,5432419,7101821912,"Ash, Stephen R.",113166379,Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette,"Ash, Stephen R.","Ash, Stephen R.",US
3,71519745,35248974600,"Saito, Hajime",60027841,Akita,Japan,Akita University,SAITO HAJIME,None,SAITO HAJIME,齊藤 元,
4,32196003,7003592507,"Mitobe, Kazutaka",60027841,Akita,Japan,Akita University,MITOBE KAZUTAKA,None,MITOBE KAZUTAKA,MITOBE KAZUTAKA,
...,...,...,...,...,...,...,...,...,...,...,...,...
146392,12621324,7005818600,"Grifo, Jamie A.",60021784,New York,United States,New York University,"GRIFO, JAMIE",None,"GRIFO, JAMIE","GRIFO, JAMIE",US
146393,29246630,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.",None,"LEESE, HENRY, J.","LEESE, HENRY, J.",GB
146394,369337,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ...","LEESE, Henry, J.","LEESE, Henry, J.",GB
146395,29246631,7005625766,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None,"HOUGHTON, FRANCHESCA, D.","HOUGHTON, FRANCHESCA, D.",GB


In [118]:
df_match_result_with_address_v2_disambiguation=df_match_result_with_address_v2.groupby(['author_id','affiliation_country']).agg(author_name=('author_name',lambda x:list(set(list(x)))),
                                                                                 psn_name=('psn_name',lambda x:list(set(list(x)))),
                                                                                 psn_name_ori=('person_name_orig_lg',lambda x:list(set(list(x)))),
                                                                                author_name_number=('author_name',lambda x:len(set(x))),
                                                                                psn_name_number=('psn_name',lambda x:len(set(x))),).reset_index()
df_match_result_with_address_v2_disambiguation

,author_id,affiliation_country,author_name,psn_name,psn_name_ori,author_name_number,psn_name_number
0,10038841100,United States,"[Wee, Han Ang]","[ANG, WEE HAN]","[Ang, Wee Han]",1,1
1,10038945100,China,"[Ma, Jianying]",[MA JIANYING],[马剑英],1,1
2,10039078400,United States,"[Kraus, Thomas]","[KRAUS, THOMAS]","[KRAUS, THOMAS]",1,1
3,10039332100,China,"[Ji, Xiang]",[JI XIANG],[计翔],1,1
4,10039611300,China,"[Chen, Yingwen]",[CHEN YINGWEN],[CHEN YINGWEN],1,1
...,...,...,...,...,...,...,...
67735,9942054000,South Korea,"[Jung, Hyungil]","[JUNG, HYUNGIL, JUNG HYUNGIL]","[JUNG, Hyungil, JUNG HYUNGIL]",1,2
67736,9942129300,Japan,"[Miyagi, Morio]",[MORIO MIYAGI],[MORIO MIYAGI],1,1
67737,9942176400,Canada,"[Stubbs, Keith A.]","[STUBBS, KEITH]","[STUBBS, Keith, Stubbs, Keith, STUBBS, KEITH]",1,1
67738,9942918600,China,"[Shuai, Cijun]",[SHUAI CIJUN],[SHUAI CIJUN],1,1


In [129]:
Paper_Patent_Pair_v2_person_tls206[Paper_Patent_Pair_v2_person_tls206['person_name_orig_lg']=='胡志东']

,person_id,person_name,person_name_orig_lg,person_address,person_ctry_code,psn_id,psn_name,psn_sector
5748,60396175,XIAO-YONG FAN,胡志东,None,,35815389,XIAO-YONG FAN,None


In [163]:
len(df_match_result_with_address_v2['person_id'].unique())

146397

In [170]:
v1_person_id=list(df['person_id'].unique())
v2_person_id=list(df_match_result_with_address_v2['person_id'].unique())

In [173]:
len([i for i in v1_person_id if i in v2_person_id])# 被修订

91632

In [174]:
len([i for i in v2_person_id if i not in v1_person_id])# 被增加

54765

# 对数据清洗

In [124]:
df_match_result_with_address_v2_disambiguation['Chinese']=df_match_result_with_address_v2_disambiguation['psn_name_ori'].apply(lambda x:keep_pure_chinese_only(x))
df_match_result_with_address_v2_disambiguation['Chinese_number']=df_match_result_with_address_v2_disambiguation['Chinese'].apply(lambda x:len(set(list(x))))
df_match_result_with_address_v2_disambiguation

,author_id,affiliation_country,author_name,psn_name,psn_name_ori,author_name_number,psn_name_number,Chinese,Chinese_number
0,10038841100,United States,"[Wee, Han Ang]","[ANG, WEE HAN]","[Ang, Wee Han]",1,1,[],0
1,10038945100,China,"[Ma, Jianying]",[MA JIANYING],[马剑英],1,1,[马剑英],1
2,10039078400,United States,"[Kraus, Thomas]","[KRAUS, THOMAS]","[KRAUS, THOMAS]",1,1,[],0
3,10039332100,China,"[Ji, Xiang]",[JI XIANG],[计翔],1,1,[计翔],1
4,10039611300,China,"[Chen, Yingwen]",[CHEN YINGWEN],[CHEN YINGWEN],1,1,[],0
...,...,...,...,...,...,...,...,...,...
67735,9942054000,South Korea,"[Jung, Hyungil]","[JUNG, HYUNGIL, JUNG HYUNGIL]","[JUNG, Hyungil, JUNG HYUNGIL]",1,2,[],0
67736,9942129300,Japan,"[Miyagi, Morio]",[MORIO MIYAGI],[MORIO MIYAGI],1,1,[],0
67737,9942176400,Canada,"[Stubbs, Keith A.]","[STUBBS, KEITH]","[STUBBS, Keith, Stubbs, Keith, STUBBS, KEITH]",1,1,[],0
67738,9942918600,China,"[Shuai, Cijun]",[SHUAI CIJUN],[SHUAI CIJUN],1,1,[],0


In [132]:
df_match_result_with_address_v2_disambiguation[(df_match_result_with_address_v2_disambiguation['affiliation_country']=='China')
                                              &(df_match_result_with_address_v2_disambiguation['Chinese_number']>1)
                                              ].to_csv('data/version/version_v2/Chinese_Inventor_for_Revise.csv',encoding='utf-8-sig',index=None)

In [125]:
df_match_result_with_address_v2_disambiguation[df_match_result_with_address_v2_disambiguation['Chinese_number']>1]

,author_id,affiliation_country,author_name,psn_name,psn_name_ori,author_name_number,psn_name_number,Chinese,Chinese_number
126,10639053100,United States,"[Liu, Tongyao]","[LIU, Tongyao, LIU, TONGYAO, LIU TONGYAO, TONG...","[刘同耀, LIU TONGYAO, LIU, Tongyao, Liu, Tongyao,...",1,4,"[刘同耀, 刘彤瑶]",2
195,11339198400,China,"[Fan, Xiaoyong]","[FAN, XIAO-YONG, XIAO-YONG FAN]","[FAN, Xiao-Yong, Fan, Xiao-Yong, 胡志东, 范小勇]",1,2,"[胡志东, 范小勇]",2
326,12242175100,China,"[Cai, Qing]","[CAI QING, QING CAI]","[艾淼, 蔡晴]",1,2,"[艾淼, 蔡晴]",2
344,12344907000,China,"[Yang, Yan]",[YANG YAN],"[杨妍, 杨焱]",1,1,"[杨妍, 杨焱]",2
604,12799314000,China,"[Wang, Yue]",[WANG YUE],"[王跃, 王悦]",1,1,"[王跃, 王悦]",2
...,...,...,...,...,...,...,...,...,...
66596,8679878600,China,"[Yu, Jun]","[YU, JUN]","[沈祖尧, 梁巧仪, YU, JUN, YU, Jun]",1,1,"[沈祖尧, 梁巧仪]",2
66635,8690432600,Canada,"[Ogawa, Shinichiro]","[OGAWA SHINICHIRO, OGAWA, SHINICHIRO]","[OGAWA, SHINICHIRO, 小川新一郎, OGAWA, Shinichiro, ...",1,2,"[小川新一郎, 小川新郎]",2
66944,8859694900,China,"[Li, Jipeng]",[LI JIPENG],"[李吉鹏, LI JIPENG, 李纪鹏]",1,1,"[李吉鹏, 李纪鹏]",2
67645,9744814600,China,"[Zhang, Chang Qing]",[ZHANG QING],"[张青, 张清]",1,1,"[张青, 张清]",2


In [134]:
Chinese_Inventor_for_Revise_Done=pd.read_csv('data/version/version_v2/Chinese_Inventor_for_Revised_Done.csv')
Chinese_Inventor_for_Revise_Done

,author_id,affiliation_country,author_name,psn_name,psn_name_ori,author_name_number,psn_name_number,Chinese,Chinese_number,需要删除
0,11339198400,China,"['Fan, Xiaoyong']","['FAN, XIAO-YONG', 'XIAO-YONG FAN']","['FAN, Xiao-Yong', 'Fan, Xiao-Yong', '胡志东', '范...",1,2,"['胡志东', '范小勇']",2,胡志东
1,12242175100,China,"['Cai, Qing']","['CAI QING', 'QING CAI']","['艾淼', '蔡晴']",1,2,"['艾淼', '蔡晴']",2,艾淼
2,23392840800,China,"['Hai, Li']","['HAI LI', 'LI HAI']","['彭瑶', '海俐']",1,2,"['彭瑶', '海俐']",2,彭瑶
3,25625152200,China,"['Jiao, Li Li']","['JIAO LILI', 'LILI JIAO']","['焦丽丽', '李慧']",1,2,"['焦丽丽', '李慧']",2,李慧
4,36447018100,China,"['Zhang, Jianhua']","['ZHANG JIANHUA', 'ZHANG JIAN']","['ZHANG, JIANHUA', '张见', '张建华']",1,2,"['张见', '张建华']",2,张见
5,36986352400,China,"['Guo, Qianping']","['GUO QIYING', 'GUO QIANPING']","['过倩萍', '郭琪莹']",1,2,"['过倩萍', '郭琪莹']",2,郭琪莹
6,55265957100,China,"['Zhang, Huawei']","['ZHANG, HUAWEI', 'HUAWEI ZHANG', 'ZHANG HUAWEI']","['张华伟', 'ZHANG, Huawei', 'HUAWEI ZHANG', '姬祥',...",1,3,"['张华伟', '姬祥']",2,姬祥
7,55475934200,China,"['Zhou, Weichang']","['ZHOU, WEICHANG', 'ZHOU, Weichang', 'ZHOU WEI...","['周伟昌', '唐思远', 'ZHOU WEICHANG', 'ZHOU, WEICHAN...",1,3,"['周伟昌', '唐思远']",2,唐思远
8,55567360900,China,"['Dai, Yifan']","['Dai, Yifan', 'DAI YIFAN', 'DAI, YIFAN']","['Dai, Yifan', '戴凡', '戴一凡']",1,3,"['戴凡', '戴一凡']",2,戴凡
9,55614029700,China,"['Xu, Guoxiong']","['XU, GUOTONG', 'XU GUOXIONG']","['许国雄', '徐国彤']",1,2,"['许国雄', '徐国彤']",2,徐国彤


In [156]:
del_name=chain(*Chinese_Inventor_for_Revise_Done['需要删除'].str.split(';'))
Paper_Patent_Pair_v2_person_tls206_Chinese=Paper_Patent_Pair_v2_person_tls206[Paper_Patent_Pair_v2_person_tls206['person_name_orig_lg'].isin(del_name)]
Paper_Patent_Pair_v2_person_tls206_Chinese

,person_id,person_name,person_name_orig_lg,person_address,person_ctry_code,psn_id,psn_name,psn_sector
2165,65029410,WANG LI,王黎,None,,34568843,WANG LI,INDIVIDUAL
2893,65483547,Qing Cai,艾淼,None,,26269761,QING CAI,None
3206,76106479,"DUAN, Hongmin",段洪敏,None,CN,7391468,"DUAN, HONGMIN",None
5355,67071432,WEN ZUORUI,文作瑞,None,,34915492,WEN ZUORUI,None
5356,66275303,ZUORUI WEN,牛学良,None,,37585136,ZUORUI WEN,None
5358,60669643,WEI SUN,文作瑞,None,,34801480,WEI SUN,None
5405,65557597,ZHANG QI,张琪,None,,37069017,ZHANG QI,INDIVIDUAL
5748,60396175,XIAO-YONG FAN,胡志东,None,,35815389,XIAO-YONG FAN,None
6331,73194224,LILI JIAO,李慧,None,,19484266,LILI JIAO,None
13847,65808865,LI HUI,李慧,None,,19303111,LI HUI,INDIVIDUAL


In [184]:
Paper_Patent_Pair_v2_person_tls206_Chinese.to_pickle('data/version/version_v2/Paper_Patent_Pair_v2_person_tls206_Chinese.pickle')

In [155]:
Counter(df_match_result_with_address_v2_disambiguation['affiliation_country'])['China']/146357 

0.16061411480147858

# western country

In [154]:
df_match_result_with_address_v2_disambiguation_American=df_match_result_with_address_v2_disambiguation[(df_match_result_with_address_v2_disambiguation['affiliation_country'].isin(['United States','United Kingdom']))
                                              &(df_match_result_with_address_v2_disambiguation['psn_name_number']>1)]
df_match_result_with_address_v2_disambiguation_American

,author_id,affiliation_country,author_name,psn_name,psn_name_ori,author_name_number,psn_name_number,Chinese,Chinese_number
11,10040453400,United States,"[Alper, Hal]","[ALPER HAL S, ALPER, HAL, S.]","[ALPER HAL S, ALPER, Hal, S., ALPER, HAL, S.]",1,2,[],0
15,10041366400,United States,"[Gray, John T.]","[GRAY, JOHN T., GRAY, JOHN, T.]","[GRAY, John, T., GRAY, JOHN, T., GRAY, JOHN T.]",1,2,[],0
17,10041688800,United States,"[Wilusz, Jeremy E.]","[WILUSZ, JEREMY, E., WILUSZ, JEREMY E.]","[WILUSZ, Jeremy, E., WILUSZ, JEREMY, E., WILUS...",1,2,[],0
18,10041768000,United States,"[Zhan, Yiqiang]","[ZHAN YIQIANG, ZHAN, YIQIANG]","[Zhan, Yiqiang, 詹翊强]",1,2,[詹翊强],1
29,10042887200,United States,"[Krastins, Bryan]","[KRASTINS BRYAN, KRASTINS, BRYAN]","[KRASTINS, Bryan, KRASTINS BRYAN, KRASTINS, BR...",1,2,[],0
...,...,...,...,...,...,...,...,...,...
67678,9838911500,United States,"[Issafras, Hassan]","[ISSAFRAS HASSAN, ISSAFRAS, HASSAN]","[Issafras, Hassan, ISSAFRAS, HASSAN, 이사프라스 하산,...",1,2,[],0
67690,9841399300,United States,"[Abate, Adam R.]","[ABATE, ADAM R., ABATE, ADAM, R.]","[Abate, Adam R., ABATE, ADAM R., ABATE, Adam, ...",1,2,[],0
67697,9842876400,United States,"[Christensen, Lane V.]","[CHRISTENSEN LANE V, CHRISTENSEN, LANE, V., CH...","[Christensen, Lane V., CHRISTENSEN LANE V., CH...",1,3,[],0
67699,9843585800,United Kingdom,"[Nagy, Z.]","[NAGY, ZSUZSANNA, ZSUZSANNA NAGY]","[Nagy, Zsuzsanna, ZSUZSANNA NAGY, NAGY, Zsuzsa...",1,2,[],0


In [ ]:
df_match_result_with_address_v2_disambiguation_American.to_csv('data/version/version_v2/American_Inventor_for_Revise.csv',encoding='utf-8-sig',index=None)

# 生成github

In [3]:
df_match_result_with_address=pd.read_pickle('data/version/version_v2/Paper_Patent_Pair_v2.pickle')
df_match_result_with_address

,person_id,author_id,author_name,afid,affiliation_city,affiliation_country,affilname,psn_name,person_address
1,5762813,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison
2,37692963,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None
3,5432419,7101821912,"Ash, Stephen R.",113166379,Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette
4,71519745,35248974600,"Saito, Hajime",60027841,Akita,Japan,Akita University,SAITO HAJIME,None
5,32196003,7003592507,"Mitobe, Kazutaka",60027841,Akita,Japan,Akita University,MITOBE KAZUTAKA,None
...,...,...,...,...,...,...,...,...,...
146565,12621324,7005818600,"Grifo, Jamie A.",60021784,New York,United States,New York University,"GRIFO, JAMIE",None
146566,29246630,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.",None
146567,369337,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ..."
146568,29246631,7005625766,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None


In [4]:
Paper_Patent_Pair_v2_person_tls206_Chinese=pd.read_pickle('data/version/version_v2/Paper_Patent_Pair_v2_person_tls206_Chinese.pickle')
Paper_Patent_Pair_v2_person_tls206_Chinese

,person_id,person_name,person_name_orig_lg,person_address,person_ctry_code,psn_id,psn_name,psn_sector
2165,65029410,WANG LI,王黎,None,,34568843,WANG LI,INDIVIDUAL
2893,65483547,Qing Cai,艾淼,None,,26269761,QING CAI,None
3206,76106479,"DUAN, Hongmin",段洪敏,None,CN,7391468,"DUAN, HONGMIN",None
5355,67071432,WEN ZUORUI,文作瑞,None,,34915492,WEN ZUORUI,None
5356,66275303,ZUORUI WEN,牛学良,None,,37585136,ZUORUI WEN,None
5358,60669643,WEI SUN,文作瑞,None,,34801480,WEI SUN,None
5405,65557597,ZHANG QI,张琪,None,,37069017,ZHANG QI,INDIVIDUAL
5748,60396175,XIAO-YONG FAN,胡志东,None,,35815389,XIAO-YONG FAN,None
6331,73194224,LILI JIAO,李慧,None,,19484266,LILI JIAO,None
13847,65808865,LI HUI,李慧,None,,19303111,LI HUI,INDIVIDUAL


In [6]:
df_match_result_with_address=df_match_result_with_address[~df_match_result_with_address['person_id'].isin(Paper_Patent_Pair_v2_person_tls206_Chinese['person_id'])]
df_match_result_with_address

,person_id,author_id,author_name,afid,affiliation_city,affiliation_country,affilname,psn_name,person_address
1,5762813,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison
2,37692963,7005737279,"Robins, H. I.",60032179,Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None
3,5432419,7101821912,"Ash, Stephen R.",113166379,Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette
4,71519745,35248974600,"Saito, Hajime",60027841,Akita,Japan,Akita University,SAITO HAJIME,None
5,32196003,7003592507,"Mitobe, Kazutaka",60027841,Akita,Japan,Akita University,MITOBE KAZUTAKA,None
...,...,...,...,...,...,...,...,...,...
146565,12621324,7005818600,"Grifo, Jamie A.",60021784,New York,United States,New York University,"GRIFO, JAMIE",None
146566,29246630,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.",None
146567,369337,7006373192,"Leese, H. J.",60016418,York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ..."
146568,29246631,7005625766,"Houghton, F. D.",60025225,Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None


In [10]:
df_github=df_match_result_with_address[['author_name', 'affiliation_city','affiliation_country','affilname','psn_name','person_address']]

# df_github=df_github.drop(['appln_id'],axis=1)
df_github.to_csv('GitHub/Inventor_Scholar_Matching.csv',encoding='utf-8-sig',index=None)
df_github

,author_name,affiliation_city,affiliation_country,affilname,psn_name,person_address
1,"Robins, H. I.",Madison,United States,University of Wisconsin-Madison,"ROBINS, H. IAN",Madison
2,"Robins, H. I.",Madison,United States,University of Wisconsin-Madison,"ROBINS, H., IAN",None
3,"Ash, Stephen R.",Lafayette,United States,"Greater Lafayette Health Services, Inc.","ASH, STEPHEN R.",Lafayette
4,"Saito, Hajime",Akita,Japan,Akita University,SAITO HAJIME,None
5,"Mitobe, Kazutaka",Akita,Japan,Akita University,MITOBE KAZUTAKA,None
...,...,...,...,...,...,...
146565,"Grifo, Jamie A.",New York,United States,New York University,"GRIFO, JAMIE",None
146566,"Leese, H. J.",York,United Kingdom,University of York,"LEESE, HENRY, J.",None
146567,"Leese, H. J.",York,United Kingdom,University of York,"LEESE, HENRY, J.","Department of Biology University of York,York ..."
146568,"Houghton, F. D.",Southampton,United Kingdom,University of Southampton,"HOUGHTON, FRANCHESCA, D.",None
